In [1]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

# Georgia

## Election Results Documentation:

Election results from the Georgia Secretary of State Elections Division (https://sos.ga.gov/index.php/Elections/current_and_past_elections_results). Presidential recount results from the Georgia Secretary of State Elections Division via Reuters.

## Shapefile Source Documentation:

Precinct shapefile primarily from the Georgia General Assembly Reapportionment Office (http://www.legis.ga.gov/Joint/reapportionment/en-US/default.aspx).  

The following counties instead use shapefiles from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release: Cobb, DeKalb, Gwinnett. Forsyth and Fulton use shapefiles sourced from the respective counties.

## Shapefile Modifications Documentation:

Three of the four VTDs in Chattahoochee County are comprised of Fort Benning. However, the county only reports one polling location for all voters, including residents of Fort Benning that vote within the county. The four Chattahoochee County VTDs have therefore been merged in the shapefile.

The following additional modifications reflect changes made prior to the 2020 general election:

Barrow: Merge 2/15, 3/12, 4/14, 5/7, 6/10/13, 8/9, 11/16; Adjust new 2/13 boundary
Bartow: Split Cassville/Hamilton Crossing
Candler: Merge Candler/Metter as Jack Strickland Comm Center
Chatham: Split 7-7/8-16, 7-12/7-16; Realign 7-06C/7-07C
Chatooga: Split Cloudland/Teloga along ridgeline that marks boundary between them with the USGS Topographic Contour shapefile
Clayton: Split Ellenswood 1/2, Jonesboro 1/17/19, Lovejoy 3/6/7, Morrow 3/11, 5/10, Oak 3/5 
Cobb: Split Bells Ferry 3/4, Dobbins 1/2, Marietta 3A/3B, Smyrna 3A/3B
Columbia: Split Bessie Thomas/2nd Mt Moriah, Harlem Branch/Harlem Senior Ctr; Merge Blanchard Park/MTZ Col FD;  Align multiple precincts with county maps
Coweta: Merge Arts Centre/Jefferson Parkway as Newnan Centre
Fulton: Merge CP07A/CP07D, CH01/CH04B, SS29A/SS29B, UC031/UC035
DeKalb: Split Clarkston/Clarkston Comm Ctr; Realign Decatur/Oakhurst; Align precincts with Atlanta, Brookhaven, Decatur, Tucker city limits 
Gwinnett: Adjust Baycreek F/G, Berkshire J/M, Cates D/F, Garners C/B, Lawrenceville G/N, Pinckneyville S/T, Rockbridge A/G
Lowndes: Split Northgate Assembly/Trinity, Jaycee/Mt Calvary/Northside/VSU
Oconee: Merge Annex/City Hall; Align City Hall with Watkinsville city limits
Paulding: Reorganize 12 precincts into 19 precincts as redrawn in 2019
Randolph: Merge Carnegie/Cuthbert-Courthouse, 4th District/Fountain Bridge/Shellman
Troup: Split Mountville between Gardner Newman/Hogansville/Rosemont; Align multiple precincts with county maps
Towns: Merge Macedonia/Tate City

Note that the leading zeros in the Paulding County precinct IDs are included in some election reports and omitted in others. The shapefile includes the leading zeros consistent with the voter file.

## Candidate List

G20PRERTRU - Donald J. Trump (Republican Party)  
G20PREDBID - Joseph R. Biden (Democratic Party)  
G20PRELJOR - Jo Jorgensen (Libertarian Party)  
  
C20PRERTRU - Donald J. Trump (Republican Party)  
C20PREDBID - Joseph R. Biden (Democratic Party)  
C20PRELJOR - Jo Jorgensen (Libertarian Party)  
  
G20USSRPER - David A. Perdue (Republican Party)  
G20USSDOSS - Jon Ossoff (Democratic Party)  
G20USSLHAZ - Shane Hazel (Libertarian Party)  
  
S20USSRLOE - Kelly Loeffler (Republican Party)  
S20USSRCOL - Doug Collins (Republican Party)  
S20USSRGRA - Derrick E. Grayson (Republican Party)  
S20USSRJAC - Annette Davis Jackson (Republican Party)  
S20USSRTAY - Kandiss Taylor (Republican Party)  
S20USSRJOH - A. Wayne Johnson (Republican Party)  
S20USSDWAR - Raphael Warnock (Democratic Party)  
S20USSDJAC - Deborah Jackson (Democratic Party)  
S20USSDLIE - Matt Lieberman (Democratic Party)  
S20USSDJOH - Tamara Johnson-Shealey (Democratic Party)  
S20USSDJAM - Jamesia James (Democratic Party)  
S20USSDSLA - Joy Felicia Slade (Democratic Party)  
S20USSDWIN - Richard Dien Winfield (Democratic Party)  
S20USSDTAR - Ed Tarver (Democratic Party)  
S20USSLSLO - Brian Slowinski (Libertarian Party)  
S20USSGFOR - John Fortuin (Green Party)  
S20USSIBUC - Allen Buckley (Independent)  
S20USSIBAR - Al Bartell (Independent)  
S20USSISTO - Valencia Stovall (Independent)  
S20USSIGRE - Michael Todd Greene (Independent)  
  
G20PSCRSHA - Jason Shaw (Republican Party)  
G20PSCDBRY - Robert G. Bryant (Democratic Party)  
G20PSCLMEL - Elizabeth Melton (Libertarian Party)  
  
G20PSCRMCD - Lauren Bubba McDonald, Jr. (Republican Party)  
G20PSCDBLA - Daniel Blackman (Democratic Party)  
G20PSCLWIL - Nathan Wilson (Libertarian Party)  

### Load Non-Recount Election Data

This data had to be downloaded county-by-county in XML format. The below code parses the XML and grabs the necessary data, adds it to a list, gives it the appropriate column names, and converts the data into a dataframe.

In [2]:
loaded_counties = os.listdir("./raw-from-source/Non_Recount_Results/")
z=[]
for locale in loaded_counties:
    if locale.endswith('.xml'):
        file_string = "./raw-from-source/Non_Recount_Results/"+locale
        xtree = et.parse(file_string)
        xroot = xtree.getroot()
        store_list = []
        county_area = xroot.findall(".//Region")
        for i in county_area:
            county = i.text
        contests = xroot.findall(".//Contest")
        for i in contests:
            contest = i.attrib.get('text')
            lower = i.findall("./Choice")
            for j in lower:
                choice = j.attrib.get('text')
                lower_2 = j.findall("./VoteType")
                for k in lower_2:
                    voting_method = k.attrib.get('name')
                    lower_3 = k.findall("./Precinct")
                    for l in lower_3:
                        precinct_name = l.attrib.get('name')
                        num_votes = l.attrib.get('votes')
                        z.append([county,contest,choice,voting_method,precinct_name,num_votes])
dfcols = ['county','contest','choice','voting_method','precinct','num_votes']
df_general = pd.DataFrame(z,columns=dfcols)

#### Take a look at the dataframe

In [3]:
df_general.head(1)

,county,contest,choice,voting_method,precinct,num_votes
0,Crisp,President of the United States,Donald J. Trump (I) (Rep),Election Day Votes,Arabi,163


### Load Recount Election Data

This data had to be downloaded county-by-county in XML format. The below code parses the XML and grabs the necessary data, adds it to a list, gives it the appropriate column names, and converts the data into a dataframe.

In [4]:
loaded_counties = os.listdir("./raw-from-source/Recount_Results/")
z=[]
for locale in loaded_counties:
    if locale.endswith('.xml'):
        file_string = "./raw-from-source/Recount_Results/"+locale
        xtree = et.parse(file_string)
        xroot = xtree.getroot()
        store_list = []
        county_area = xroot.findall(".//Region")
        for i in county_area:
            county = i.text
        contests = xroot.findall(".//Contest")
        for i in contests:
            contest = i.attrib.get('text')
            lower = i.findall("./Choice")
            for j in lower:
                choice = j.attrib.get('text')
                lower_2 = j.findall("./VoteType")
                for k in lower_2:
                    voting_method = k.attrib.get('name')
                    lower_3 = k.findall("./Precinct")
                    for l in lower_3:
                        precinct_name = l.attrib.get('name')
                        num_votes = l.attrib.get('votes')
                        z.append([county,contest,choice,voting_method,precinct_name,num_votes])
dfcols = ['county','contest','choice','voting_method','precinct','num_votes']
df_recount = pd.DataFrame(z,columns=dfcols)

#### Take a look at the dataframe

In [5]:
df_recount.head(2)

,county,contest,choice,voting_method,precinct,num_votes
0,Oconee,President of the United States,Donald J. Trump (I) (Rep),Election Day Votes,City Hall,244
1,Oconee,President of the United States,Donald J. Trump (I) (Rep),Election Day Votes,Colham Ferry,182


### Clean Recount Data - Contest Names

Note: Doing this before combining with the other data so the presidential vote tallies don't get mixed up

In [6]:
print(df_recount["contest"].unique())

contest_changes_dict = {'President of the United States':'President-Recount',
 'President of the United States/Presidentede los Estados Unidos':'President-Recount'}

df_recount["contest"] = df_recount["contest"].map(contest_changes_dict).fillna(df_recount["contest"])

['President of the United States'
 'President of the United States/Presidentede los Estados Unidos']


### Join Election Data Together

In [7]:
ga_election = pd.concat([df_general,df_recount])

#Sanity check that there are the right number of counties, should be 159
print(len(ga_election["county"].unique()))

159


### Filter Down to Relevant Races

In [8]:
office_list = ['President of the United States','US Senate (Perdue)','US Senate (Loeffler) - Special',
              'Public Service Commission District 1','Public Service Commission District 4',
              'US Senate (Loeffler) - Special Election','President-Recount','President of the United States/Presidentede los Estados Unidos',
              'Public Service Commission Dist 1/Comisionado de Servicio Público Dist 1',
       'Public Service Commission Dist 4/Comisionado de Servicio Público Dist 4','US Senate (Perdue)/Senado de los EE.UU. (Perdue)',
       'US Senate (Loeffler) - Special/Senado de los EE.UU. (Loeffler) - Especial',]

ga_election = ga_election[ga_election["contest"].isin(office_list)]

### Clean and Pivot Data

In [9]:
print(ga_election["contest"].unique())

['President of the United States' 'US Senate (Perdue)'
 'US Senate (Loeffler) - Special' 'Public Service Commission District 1'
 'Public Service Commission District 4'
 'US Senate (Loeffler) - Special Election'
 'President of the United States/Presidentede los Estados Unidos'
 'US Senate (Perdue)/Senado de los EE.UU. (Perdue)'
 'US Senate (Loeffler) - Special/Senado de los EE.UU. (Loeffler) - Especial'
 'Public Service Commission Dist 1/Comisionado de Servicio Público Dist 1'
 'Public Service Commission Dist 4/Comisionado de Servicio Público Dist 4'
 'President-Recount']


In [10]:
contest_changes_dict = {'US Senate (Loeffler) - Special Election':'US Senate (Loeffler) - Special',
                        'US Senate (Loeffler) - Special/Senado de los EE.UU. (Loeffler) - Especial':'US Senate (Loeffler) - Special',
                        'US Senate (Perdue)/Senado de los EE.UU. (Perdue)':'US Senate (Perdue)',
                        'President of the United States/Presidentede los Estados Unidos':'President of the United States',
                        'Public Service Commission Dist 1/Comisionado de Servicio Público Dist 1':'Public Service Commission District 1',
                        'Public Service Commission Dist 4/Comisionado de Servicio Público Dist 4':'Public Service Commission District 4'}

ga_election["contest"] = ga_election["contest"].map(contest_changes_dict).fillna(ga_election["contest"])

#### Check remaining contests

In [11]:
print(ga_election["contest"].unique())

['President of the United States' 'US Senate (Perdue)'
 'US Senate (Loeffler) - Special' 'Public Service Commission District 1'
 'Public Service Commission District 4' 'President-Recount']


### Clean Candidate Names

In [12]:
print(ga_election["choice"].unique())

['Donald J. Trump (I) (Rep)' 'Joseph R. Biden (Dem)' 'Jo Jorgensen (Lib)'
 'David A. Perdue (I) (Rep)' 'Jon Ossoff (Dem)' 'Shane Hazel (Lib)'
 'Al Bartell (Ind)' 'Allen Buckley (Ind)' 'Doug Collins (Rep)'
 'John Fortuin (Grn)' 'Derrick E. Grayson (Rep)'
 'Michael Todd Greene (Ind)' 'Annette Davis Jackson (Rep)'
 'Deborah Jackson (Dem)' 'Jamesia James (Dem)' 'A. Wayne Johnson (Rep)'
 'Tamara Johnson-Shealey (Dem)' 'Matt Lieberman (Dem)'
 'Kelly Loeffler (I) (Rep)' 'Joy Felicia Slade (Dem)'
 'Brian Slowinski (Lib)' 'Valencia Stovall (Ind)' 'Ed Tarver (Dem)'
 'Kandiss Taylor (Rep)' 'Raphael Warnock (Dem)'
 'Richard Dien Winfield (Dem)' 'Jason Shaw (I) (Rep)'
 'Robert G. Bryant (Dem)' 'Elizabeth Melton (Lib)'
 'Lauren Bubba McDonald, Jr. (I) (Rep)' 'Daniel Blackman (Dem)'
 'Nathan Wilson (Lib)' 'Matt Lierberman (Dem)']


#### Correct the Candidate Name Typo

In [13]:
cand_change_dict = {'Matt Lierberman (Dem)':'Matt Lieberman (Dem)'}
ga_election["choice"] = ga_election["choice"].map(cand_change_dict).fillna(ga_election["choice"])

In [14]:
ga_election["num_votes"] = ga_election["num_votes"].astype(int)

#### Add a FIPS Column

In [15]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Georgia"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
fips_file["County Name"] = fips_file["County Name"].replace("De Kalb","DeKalb")
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))
ga_election['county_fips'] = ga_election['county'].map(fips_dict).fillna(ga_election['county'])
ga_election['county_fips'] = ga_election['county_fips'].astype(str)
ga_election['county_fips'] = ga_election['county_fips'].str.zfill(3)

#### Pivot the data

In [16]:
ga_election["unique_ID"]=ga_election["county_fips"]+ga_election["precinct"]
ga_election["choice"]=ga_election["choice"]+ga_election["contest"]
ga_election=pd.pivot_table(ga_election,index=["unique_ID","county","county_fips","precinct"],columns=["choice"],values=['num_votes'],aggfunc=sum)
ga_election = ga_election.fillna(0)
ga_election.columns = ga_election.columns.droplevel(0)
ga_election = ga_election.reset_index()

#### Change Names to match VEST

In [17]:
column_changes_dict = {'A. Wayne Johnson (Rep)US Senate (Loeffler) - Special':'S20USSRJOH',
       'Al Bartell (Ind)US Senate (Loeffler) - Special':'S20USSIBAR',
       'Allen Buckley (Ind)US Senate (Loeffler) - Special':'S20USSIBUC',
       'Annette Davis Jackson (Rep)US Senate (Loeffler) - Special':'S20USSRJAC',
       'Brian Slowinski (Lib)US Senate (Loeffler) - Special':'S20USSLSLO',
       'Daniel Blackman (Dem)Public Service Commission District 4':'G20PSCDBLA',
       'David A. Perdue (I) (Rep)US Senate (Perdue)':'G20USSRPER',
       'Deborah Jackson (Dem)US Senate (Loeffler) - Special':'S20USSDJAC',
       'Derrick E. Grayson (Rep)US Senate (Loeffler) - Special':'S20USSRGRA',
       'Donald J. Trump (I) (Rep)President of the United States':'G20PRERTRU',
       'Donald J. Trump (I) (Rep)President-Recount':'C20PRERTRU',
       'Doug Collins (Rep)US Senate (Loeffler) - Special':'S20USSRCOL',
       'Ed Tarver (Dem)US Senate (Loeffler) - Special':'S20USSDTAR',
       'Elizabeth Melton (Lib)Public Service Commission District 1':'G20PSCLMEL',
       'Jamesia James (Dem)US Senate (Loeffler) - Special':'S20USSDJAM',
       'Jason Shaw (I) (Rep)Public Service Commission District 1':'G20PSCRSHA',
       'Jo Jorgensen (Lib)President of the United States':'G20PRELJOR',
       'Jo Jorgensen (Lib)President-Recount':'C20PRELJOR',
       'John Fortuin (Grn)US Senate (Loeffler) - Special':'S20USSGFOR',
       'Jon Ossoff (Dem)US Senate (Perdue)':'G20USSDOSS',
       'Joseph R. Biden (Dem)President of the United States':'G20PREDBID',
       'Joseph R. Biden (Dem)President-Recount':'C20PREDBID',
       'Joy Felicia Slade (Dem)US Senate (Loeffler) - Special':'S20USSDSLA',
       'Kandiss Taylor (Rep)US Senate (Loeffler) - Special':'S20USSRTAY',
       'Kelly Loeffler (I) (Rep)US Senate (Loeffler) - Special':'S20USSRLOE',
       'Lauren Bubba McDonald, Jr. (I) (Rep)Public Service Commission District 4':'G20PSCRMCD',
       'Matt Lieberman (Dem)US Senate (Loeffler) - Special':'S20USSDLIE',
       'Michael Todd Greene (Ind)US Senate (Loeffler) - Special':'S20USSIGRE',
       'Nathan Wilson (Lib)Public Service Commission District 4':'G20PSCLWIL',
       'Raphael Warnock (Dem)US Senate (Loeffler) - Special':'S20USSDWAR',
       'Richard Dien Winfield (Dem)US Senate (Loeffler) - Special':'S20USSDWIN',
       'Robert G. Bryant (Dem)Public Service Commission District 1':'G20PSCDBRY',
       'Shane Hazel (Lib)US Senate (Perdue)':'G20USSLHAZ',
       'Tamara Johnson-Shealey (Dem)US Senate (Loeffler) - Special':'S20USSDJOH',
       'Valencia Stovall (Ind)US Senate (Loeffler) - Special':'S20USSISTO'}

ga_election = ga_election.rename(columns=column_changes_dict)

#### Order election result columns to match VEST

In [18]:
ga_election = ga_election[['unique_ID', 'county', 'county_fips', 'precinct','G20PRERTRU', 'G20PREDBID',
       'G20PRELJOR', 'C20PRERTRU', 'C20PREDBID', 'C20PRELJOR', 'G20USSRPER',
       'G20USSDOSS', 'G20USSLHAZ', 'S20USSRLOE', 'S20USSRCOL', 'S20USSRGRA',
       'S20USSRJAC', 'S20USSRTAY', 'S20USSRJOH', 'S20USSDWAR', 'S20USSDJAC',
       'S20USSDLIE', 'S20USSDJOH', 'S20USSDJAM', 'S20USSDSLA', 'S20USSDWIN',
       'S20USSDTAR', 'S20USSLSLO', 'S20USSGFOR', 'S20USSIBUC', 'S20USSIBAR',
       'S20USSISTO', 'S20USSIGRE', 'G20PSCRSHA', 'G20PSCDBRY', 'G20PSCLMEL',
       'G20PSCRMCD', 'G20PSCDBLA', 'G20PSCLWIL']]

### Load the VEST file

In [19]:
vest_ga_20 = gp.read_file("./raw-from-source/VEST/ga_2020/ga_2020.shp")
print(vest_ga_20.shape)

(2679, 44)


#### Take a look

In [20]:
vest_ga_20.head(1)


,DISTRICT,CTYSOSID,PRECINCT_I,PRECINCT_N,CTYNAME,CTYNUMBER,CTYNUMBER2,FIPS2,G20PRERTRU,G20PREDBID,...,S20USSIBAR,S20USSISTO,S20USSIGRE,G20PSCRSHA,G20PSCDBRY,G20PSCLMEL,G20PSCRMCD,G20PSCDBLA,G20PSCLWIL,geometry
0,215122,215122,122,FIRST AFRICAN,MUSCOGEE,106,106,215,238,668,...,9,1,4,251,587,52,251,587,44,"POLYGON ((-84.96984 32.46725, -84.97031 32.467..."


In [21]:
data_columns = ['G20PRERTRU', 'G20PREDBID',
       'G20PRELJOR', 'C20PRERTRU', 'C20PREDBID', 'C20PRELJOR', 'G20USSRPER',
       'G20USSDOSS', 'G20USSLHAZ', 'S20USSRLOE', 'S20USSRCOL', 'S20USSRGRA',
       'S20USSRJAC', 'S20USSRTAY', 'S20USSRJOH', 'S20USSDWAR', 'S20USSDJAC',
       'S20USSDLIE', 'S20USSDJOH', 'S20USSDJAM', 'S20USSDSLA', 'S20USSDWIN',
       'S20USSDTAR', 'S20USSLSLO', 'S20USSGFOR', 'S20USSIBUC', 'S20USSIBAR',
       'S20USSISTO', 'S20USSIGRE', 'G20PSCRSHA', 'G20PSCDBRY', 'G20PSCLMEL',
       'G20PSCRMCD', 'G20PSCDBLA', 'G20PSCLWIL']

for race in data_columns:
    ga_election[race] = ga_election[race].astype(int)

### Compare Statewide Candidate Totals

In [22]:
no_diff = True
for race in data_columns:
    #Un-comment the below to visually check results against official state counts: https://results.enr.clarityelections.com/GA/105369/web.264614/#/summary
    #print(race+": "+str(sum(ga_election[race])))
    if (sum(vest_ga_20[race])-sum(ga_election[race]) != 0):
        print(race+" has a difference of "+str(sum(vest_ga_20[race])-sum(ga_election[race]))+" votes")
        print("\tVEST: "+str(sum(vest_ga_20[race]))+" votes")
        print("\tSOURCES: "+str(sum(ga_election[race]))+" votes")
        no_diff = False
if (no_diff):
    print("No races contain vote differences")
else:
    print("All other races are equal")

No races contain vote differences


### Compare Countywide Candidate Totals

In [23]:
no_diff = True
diff_counties=[]
for i in data_columns:
    diff = vest_ga_20.groupby(["FIPS2"]).sum()[i]-ga_election.groupby(["county_fips"]).sum()[i]
    for val in diff[diff != 0].index.values.tolist():
        if val not in diff_counties:
            diff_counties.append(val)
    if len(diff[diff != 0]!=0):
        print(i)
        print(diff[diff != 0].to_string(header=False))
        no_diff = False
if (no_diff):
    print("No counties contain vote differences")
else:
    print("All other county race totals are equal")

No counties contain vote differences


### Join Election Results to VEST (to eventually validate precinct votes)

#### Try a  unique ID

In [24]:
ga_election["precinct"] = ga_election["precinct"].str.strip().str.upper()
ga_election["unique_ID"]=ga_election["county_fips"]+"-"+ga_election["precinct"]
vest_ga_20["unique_ID"]=vest_ga_20["FIPS2"]+"-"+vest_ga_20["PRECINCT_N"]

#### Check whether there are duplicate IDs

In [25]:
ser = ga_election["unique_ID"].value_counts(dropna=False)
print(ser[ser >1])

ser = vest_ga_20["unique_ID"].value_counts(dropna=False)
print(ser[ser >1])

Series([], Name: unique_ID, dtype: int64)
NaN                        7
051-ELI WHITNEY COMPLEX    2
Name: unique_ID, dtype: int64


In [26]:
#Look into duplicate IDs - ELI WHITNEY
display(vest_ga_20[vest_ga_20["unique_ID"]=="051-ELI WHITNEY COMPLEX"])

,DISTRICT,CTYSOSID,PRECINCT_I,PRECINCT_N,CTYNAME,CTYNUMBER,CTYNUMBER2,FIPS2,G20PRERTRU,G20PREDBID,...,S20USSISTO,S20USSIGRE,G20PSCRSHA,G20PSCDBRY,G20PSCLMEL,G20PSCRMCD,G20PSCDBLA,G20PSCLWIL,geometry,unique_ID
2245,0513-15C,0513-15C,3-15C,ELI WHITNEY COMPLEX,CHATHAM,25,025,051,278,361,...,0,1,301,320,16,290,325,15,"POLYGON ((-81.02948 32.06200, -81.03000 32.060...",051-ELI WHITNEY COMPLEX
2247,0512-06C,0512-06C,2-06C,ELI WHITNEY COMPLEX,CHATHAM,25,025,051,65,335,...,2,1,72,294,25,67,301,19,"POLYGON ((-81.05333 32.07291, -81.05336 32.072...",051-ELI WHITNEY COMPLEX


In [27]:
#Assign new unique IDs, adding in PRECINCT_I to differentiate
vest_ga_20.loc[vest_ga_20["CTYSOSID"]=="0513-15C","unique_ID"] = "051-ELI WHITNEY COMPLEX 3-15C"
vest_ga_20.loc[vest_ga_20["CTYSOSID"]=="0512-06C","unique_ID"] = "051-ELI WHITNEY COMPLEX 2-06C"

In [28]:
display(vest_ga_20[vest_ga_20["unique_ID"].isna()])

,DISTRICT,CTYSOSID,PRECINCT_I,PRECINCT_N,CTYNAME,CTYNUMBER,CTYNUMBER2,FIPS2,G20PRERTRU,G20PREDBID,...,S20USSISTO,S20USSIGRE,G20PSCRSHA,G20PSCDBRY,G20PSCLMEL,G20PSCRMCD,G20PSCDBLA,G20PSCLWIL,geometry,unique_ID
1329,051XFTPU,None,None,None,None,None,None,None,0,0,...,0,0,0,0,0,0,0,0,"POLYGON ((-80.96828 32.04402, -80.96876 32.044...",NaN
1447,029FTSTEW,None,None,None,None,None,None,None,0,0,...,0,0,0,0,0,0,0,0,"MULTIPOLYGON (((-81.59263 32.10346, -81.59242 ...",NaN
2125,215FTBEN2,None,None,None,None,None,None,None,0,0,...,0,0,0,0,0,0,0,0,"POLYGON ((-84.93473 32.40556, -84.93448 32.405...",NaN
2128,215FTBEN4,None,None,None,None,None,None,None,0,0,...,0,0,0,0,0,0,0,0,"POLYGON ((-84.91564 32.42299, -84.90945 32.422...",NaN
2146,215FTBEN3,None,None,None,None,None,None,None,0,0,...,0,0,0,0,0,0,0,0,"POLYGON ((-84.93783 32.40666, -84.93765 32.406...",NaN
2229,215ZZZZZZ,None,None,None,None,None,None,None,0,0,...,0,0,0,0,0,0,0,0,"POLYGON ((-84.89892 32.45201, -84.89890 32.452...",NaN
2230,215FTBEN1,None,None,None,None,None,None,None,0,0,...,0,0,0,0,0,0,0,0,"POLYGON ((-84.89897 32.44577, -84.89896 32.449...",NaN


In [29]:
vest_ga_20.loc[vest_ga_20['unique_ID'].isna(), 'unique_ID'] = vest_ga_20['DISTRICT']

#### Confirm there are no duplicate IDs

In [30]:
ser = ga_election["unique_ID"].value_counts(dropna=False)
print(ser[ser >1])

ser = vest_ga_20["unique_ID"].value_counts(dropna=False)
print(ser[ser >1])

Series([], Name: unique_ID, dtype: int64)
Series([], Name: unique_ID, dtype: int64)


In [31]:
join_attempt_1 = pd.merge(ga_election,vest_ga_20,how="outer",on="unique_ID",indicator=True)
print(join_attempt_1["_merge"].value_counts())

both          2545
right_only     134
left_only      111
Name: _merge, dtype: int64


In [32]:
left_only = join_attempt_1[join_attempt_1["_merge"]=="left_only"]
right_only = join_attempt_1[join_attempt_1["_merge"]=="right_only"]



In [37]:
source_vest_id_changes_dict = {'051-1-17C ISLANDS CHRISTIAN CHURCH': '051-ISLANDS CHRISTIAN CHURCH', '051-7-14C COASTAL CATHEDRAL': '051-COASTAL CATHEDRAL', '051-7-07C ROTHWELL BAPTIST CHURCH': '051-ROTHWELL BAPTIST CHURCH', '051-4-07C WILMINGTON ISLAND UMC': '051-WILMINGTON ISLAND UMC', '051-4-08C WILMINGTON ISLAND PRES CHURCH': '051-WILMINGTON ISLAND PRES CHURCH', '051-1-16C FERGUSON AVE BAPTIST': '051-FERGUSON AVE BAPTIST', '051-7-11C SEVENTH DAY ADV CHURCH': '051-SEVENTH DAY ADV CHURCH', '051-7-16C POOLER RECREATION CENTER GYMNASIUM': '051-POOLER REC CENTER GYM', '051-7-08C BLOOMINGDALE COMM CTR': '051-BLOOMINGDALE COMM CTR', '051-4-13C SKIDAWAY ISLAND PRES CHURCH': '051-SKIDAWAY ISLAND PRES CHURCH', '051-6-10C STATION 3': '051-GEORGETOWN ELEMENTARY', '051-4-12C ST PETERS EPISCOPAL': '051-ST PETERS EPISCOPAL', '051-7-13C SOUTHSIDE FIRE TRNG CTR': '051-SOUTHSIDE FIRE TRNG CTR', '051-4-06C FIRST BAPTIST OF THE ISLAND': '051-FIRST BAPTIST OF THE ISLAND', '051-4-04C LIGHTHOUSE BAPTIST CHURCH': '051-LIGHTHOUSE BAPTIST CHURCH', '051-4-05C ST FRANCIS EPISCOPAL CHURCH': '051-ST FRANCIS EPISCOPAL CHURCH', '051-1-12C ISLE OF HOPE BAPTIST': '051-ISLE OF HOPE BAPTIST', '051-4-02C FRANK MURRAY COMM CENTER': '051-FRANK MURRAY COMM CENTER', '051-7-06C POOLER CITY HALL': '051-POOLER CITY HALL', '051-8-16C ROYAL CINEMAS AND IMAX': '051-ROYAL CINEMAS AND IMAX', '051-6-02C WINDSOR FOREST BAPTIST': '051-WINDSOR FOREST BAPTIST', '051-6-11C BAMBOO FARMS': '051-BAMBOO FARMS', '051-4-15C SKIDAWAY ISLAND STATE PARK': '051-SKIDAWAY ISLAND STATE PARK', '051-6-09C TRINITY LUTHERAN CHURCH': '051-TRINITY LUTHERAN CHURCH', '051-1-01C FIRST PRESBYTERIAN CHURCH': '051-FIRST PRESBYTERIAN CHURCH', '051-4-14C SKIDAWAY ISLAND BAPTIST': '051-SKIDAWAY ISLAND BAPTIST', '051-7-12C POOLER CHURCH': '051-POOLER CHURCH', '051-7-10C PROGRESSIVE REC CENTER': '051-PROGRESSIVE REC CENTER', '051-1-06C CENTRAL CHURCH OF CHRIST': '051-CENTRAL CHURCH OF CHRIST', '051-1-14C COMPASSION CHRISTIAN CHURCH': '051-COMPASSION CHRISTIAN CHURCH', '051-6-03C CRUSADER COMM CENTER': '051-CRUSADER COMM CENTER', '051-4-11C TYBEE ISLAND SCHOOL CAFE': '051-TYBEE ISLAND SCHOOL CAFE', '051-7-09C SAVANNAH HOLY C OF G': '051-SAVANNAH HOLY C OF G', '051-4-10C GUARD HOUSE COMM CTR': '051-GUARD HOUSE COMM CTR', '051-1-08C GRACE UNITED METHODIST CHURCH': '051-GRACE UNITED METHODIST CHURCH', '051-1-10C ST THOMAS EPISCOPAL CHURCH': '051-ST THOMAS EPISCOPAL CHURCH', '051-1-05C JEA BUILDING': '051-JEA BUILDING', '051-1-13C THE SANCTUARY': '051-THE SANCTUARY', '051-7-15C RICE CREEK SCHOOL': '051-RICE CREEK SCHOOL', '051-3-12C THUNDERBOLT MUNI COMPLEX': '051-THUNDERBOLT MUNI COMPLEX', '051-3-13C NEW COV 7 DAY ADV CH': '051-NEW COV 7 DAY ADV CH', '051-7-03C PB EDWARDS GYM': '051-PB EDWARDS GYM', '051-7-01C GARDEN CITY SENIOR CTR': '051-GARDEN CITY SENIOR CTR', '051-6-06C THE LIGHT CHURCH': '051-THE LIGHT CHURCH', '051-7-04C LAKE SHORE COMM CTR': '051-LAKE SHORE COMM CTR', '051-6-05C WINDSOR HALL': '051-WINDSOR HALL', '051-3-02C TEMPLE MICKVE ISRAEL': '051-TEMPLE MICKVE ISRAEL', '051-6-01C WHITE BLUFF PRESBYTERIAN': '051-WHITE BLUFF PRESBYTERIAN', '051-5-11C LARGO-TIBET ELEMENTARY': '051-LARGO-TIBET ELEMENTARY', '051-3-14C OGLETHORPE CHARTER ACADEMY': '051-OGLETHORPE CHARTER ACADEMY', '051-6-08C CHRIST MEMORIAL BAPTIST CHURCH': '051-CHRIST MEMORIAL BAPTIST CHURCH', '051-7-05C WOODLAWN BAPTIST CHURCH': '051-WOODLAWN BAPTIST CHURCH', '051-5-08C SAVANNAH PRIMITIVE BC': '051-SAVANNAH PRIMITIVE BC', '051-8-03C SILK HOPE BAPTIST CHURCH': '051-SILK HOPE BAPTIST CHURCH', '051-3-05C ALDERSGATE YOUTH CENTER': '051-ALDERSGATE YOUTH CENTER', '051-3-09C COKESBURY METHODIST': '051-COKESBURY METHODIST', '051-3-15C ELI WHITNEY COMPLEX': '051-ELI WHITNEY COMPLEX 3-15C', '051-8-13C SAVANNAH CHRISTIAN': '051-SAVANNAH CHRISTIAN', '051-5-07C STATION 1': '051-ELKS LODGE', '051-2-09C SALVATION ARMY': '051-SALVATION ARMY', '051-3-10C BIBLE BAPTIST CHURCH': '051-BIBLE BAPTIST CHURCH', '051-2-07C CHRIST COMMUNITY CHURCH': '051-CHRIST COMMUNITY CHURCH', '051-3-08C JENKINS HIGH SCHOOL': '051-JENKINS HIGH SCHOOL', '051-5-06C SEED CHURCH': '051-SEED CHURCH', '051-1-09C IMMANUEL BAPTIST CHURCH': '051-IMMANUEL BAPTIST CHURCH', '051-8-02C HELLENIC CENTER': '051-HELLENIC CENTER', '051-2-11C STILLWELL TOWERS': '051-STILLWELL TOWERS', '051-3-11C SOUTHSIDE BAPTIST CHURCH': '051-SOUTHSIDE BAPTIST CHURCH', '051-2-02C BLACKSHEAR COMMUNITY CENTER': '051-BLACKSHEAR COMMUNITY CENTER', '051-5-01C BARTLETT MIDDLE SCHOOL': '051-BARTLETT MIDDLE SCHOOL', '051-2-05C HOLY SPIRIT LUTHERAN CHURCH': '051-HOLY SPIRIT LUTHERAN CHURCH', '051-2-04C FELLOWSHIP OF LOVE CHURCH': '051-FELLOWSHIP OF LOVE CHURCH', '051-2-12C WILLIAMS COURT APTS': '051-WILLIAMS COURT APTS', '051-3-01C OLD COURTHOUSE': '051-OLD COURTHOUSE', '051-2-03C W W LAW CENTER': '051-W W LAW CENTER', '051-8-12C BEACH HIGH SCHOOL': '051-BEACH HIGH SCHOOL', '051-5-10C TATUMVILLE COMMUNITY CENTER': '051-TATUMVILLE COMMUNITY CENTER', '051-8-15C GARDEN CITY REC CENTER': '051-GARDEN CITY REC CENTER', '051-2-06C ELI WHITNEY COMPLEX': '051-ELI WHITNEY COMPLEX 2-06C', '051-3-03C SAVANNAH HIGH SCHOOL': '051-SAVANNAH HIGH SCHOOL', '051-8-01C CIVIC CENTER': '051-CIVIC CENTER', '051-3-04C FIRST AFRICAN BAPTIST CHURCH': '051-FIRST AFRICAN BAPTIST CHURCH', '051-5-05C LIBERTY CITY COMM CTR': '051-LIBERTY CITY COMM CTR', '051-5-02C SENIOR CITIZENS CENTER': '051-SENIOR CITIZENS CENTER', '051-8-08C RESUR OF OUR LORD CHURCH': '051-RESURRECTION OF OUR LORD CHURCH', '051-8-05C W BROAD ST YMCA': '051-W BROAD ST YMCA', '051-8-06C TOMPKINS REC CENTER': '051-TOMPKINS REC CENTER', '051-8-09C MOSES JACKSON CENTER': '051-MOSES JACKSON CENTER', '051-5-03C BUTLER PRESBYTERIAN CHURCH': '051-BUTLER PRESBYTERIAN CHURCH', '051-8-11C BUTLER ELEMENTARY': '051-BUTLER ELEMENTARY', '051-8-07C WOODVILLE-TOMPKINS TI': '051-WOODVILLE-TOMPKINS TI', '051-8-10C CARVER HEIGHTS COMM CTR': '051-CARVER HEIGHTS COMM CTR', '053-ACTIVITY CENTER': '053-ACTIVITY CENTER (Includes FTBEN 1-3)', '067-POWDERS SPRINGS 1A': '067-POWDER SPRINGS 1A', '067-POWDERS SPRINGS 2A': '067-POWDER SPRINGS 2A', '067-POWDERS SPRINGS 3A': '067-POWDER SPRINGS 3A', '171-CHAPPELL MILL V FD': '171-CHAPPELL MILL VFD', '275-LITTLE OCHLOCKNEE BAPTIST CHURCH': '275-LITTLE OCHLOCKNEE', '299-100': '299-District 1', '299-200A': '299-District 2A', '299-200B': '299-District 2B', '299-300': '299-1231-150B', '299-304': '299-1231-150C', '299-400': '299-1231-151', '299-404': '299-Beach-Bickley', '299-405': '299-Haywood', '299-406': '299-Jamestown', '299-407': '299-Manor', '299-408': '299-Millwood', '299-409': '299-Waresboro', '317-TIGNAL SCH LUNCH RM': '317-TIGNALL SCH LUNCH RM'}

In [38]:
ga_election["unique_ID"] = ga_election["unique_ID"].map(source_vest_id_changes_dict).fillna(ga_election["unique_ID"])

In [39]:
join_attempt_2 = pd.merge(ga_election,vest_ga_20,how="outer",on="unique_ID",indicator=True)
print(join_attempt_2["_merge"].value_counts())

both = join_attempt_2[join_attempt_2["_merge"]=="both"]
left_only = join_attempt_2[join_attempt_2["_merge"]=="left_only"]
right_only = join_attempt_2[join_attempt_2["_merge"]=="right_only"]

left_only.to_csv("./source_only.csv")
right_only.to_csv("./VEST_only.csv")

both          2656
right_only      23
left_only        0
Name: _merge, dtype: int64


In [40]:
empty_precincts = []
for index, row in right_only.iterrows():
    for race in data_columns:
        keep=True
        if (row[race+"_y"]!=0):
            keep = False
    if(keep):
        empty_precincts.append(row["DISTRICT"])
print(len(empty_precincts))  

23


In [41]:
print(empty_precincts)

['121AP01E', '121UC033', '12103P1B', '121AP12D', '121SC29B', '121CP06B', '121CP053', '121SC07B', '121RW22B', '121SC08A', '121SC21A', '121UC01C', '121CP052', '051XFTPU', '029FTSTEW', '121RW11B', '12108F2', '12112E2', '215FTBEN2', '215FTBEN4', '215FTBEN3', '215ZZZZZZ', '215FTBEN1']


In [ ]:
vest_ga_20[vest_ga_20["DISTRICT"].isin(empty_precincts)]

In [ ]:
empty_unjoined = vest_ga_20[vest_ga_20["DISTRICT"].isin(empty_precincts)]

In [ ]:
display(empty_unjoined)

In [ ]:
empty_unjoined = empty_unjoined[["unique_ID","CTYNAME","FIPS2","PRECINCT_N",'S20USSRJOH',
       'S20USSIBAR', 'S20USSIBUC', 'S20USSRJAC', 'S20USSLSLO', 'G20PSCDBLA',
       'G20USSRPER', 'S20USSDJAC', 'S20USSRGRA', 'G20PRERTRU', 'C20PRERTRU',
       'S20USSRCOL', 'S20USSDTAR', 'G20PSCLMEL', 'S20USSDJAM', 'G20PSCRSHA',
       'G20PRELJOR', 'C20PRELJOR', 'S20USSGFOR', 'G20USSDOSS', 'G20PREDBID',
       'C20PREDBID', 'S20USSDSLA', 'S20USSRTAY', 'S20USSRLOE', 'G20PSCRMCD',
       'S20USSDLIE', 'S20USSIGRE', 'G20PSCLWIL', 'S20USSDWAR', 'S20USSDWIN',
       'G20PSCDBRY', 'G20USSLHAZ', 'S20USSDJOH', 'S20USSISTO']]

empty_unjoined.columns = ga_election.columns



In [ ]:
print(empty_unjoined.columns)
print(ga_election.columns)

In [ ]:
empty_unjoined.columns = ga_election.columns

In [ ]:
display(empty_unjoined)

In [ ]:
ga_election = pd.concat([ga_election,empty_unjoined])

In [ ]:
print(vest_ga_20.loc[vest_ga_20['unique_ID'].isna()])
vest_ga_20.loc[vest_ga_20['unique_ID'].isna(), 'unique_ID'] = vest_ga_20['DISTRICT']
print(vest_ga_20.loc[vest_ga_20['unique_ID'].isna()])

In [ ]:
join_attempt_2 = pd.merge(ga_election,vest_ga_20,how="outer",on="unique_ID",indicator=True)
print(join_attempt_2["_merge"].value_counts())

#Code below was used during processing
#both = join_attempt_2[join_attempt_2["_merge"]=="both"]
#left_only = join_attempt_2[join_attempt_2["_merge"]=="left_only"]
#right_only = join_attempt_2[join_attempt_2["_merge"]=="right_only"]
#left_only.to_csv("./source_only.csv")
#right_only.to_csv("./VEST_only.csv")

### Validate election results precinct-by-precinct

In [ ]:
def validater_row (df, column_List):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            diff = abs(df.iloc[j][left_Data]-df.iloc[j][right_Data])
            if(diff >0):
                if(diff>0):
                    print(i, "{:.>72}".format(df.iloc[j]["final_join_col"]), "(S)","{:.>5}".format(int(df.iloc[j][left_Data]))," (V){:.>5}".format(int(df.iloc[j][right_Data])),"(D):{:>5}".format(int(df.iloc[j][right_Data])-int(df.iloc[j][left_Data])))           
                #print(df.iloc[j]['countypct'])
                
                diff_values.append(abs(diff))
                same = False
                if(np.isnan(diff)):
                    print("NaN value at diff is: ", df.iloc[j]["final_join_col"])
                    print(df.iloc[j][left_Data])
                    print(df.iloc[j][right_Data])
                if (diff>max_diff):
                    max_diff = diff
                    #print("New max diff is: ", str(max_diff))
                    #print(df.iloc[j]['cty_pct'])
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j]["final_join_col"])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    diff_list.sort()
    print(diff_list)
    
validater_row(both,data_columns)

## Shapefiles 

### Shapefile Sources

Precinct shapefile primarily from the Georgia General Assembly Reapportionment Office (http://www.legis.ga.gov/Joint/reapportionment/en-US/default.aspx).  

The following counties instead use shapefiles from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release: Cobb, DeKalb, Gwinnett. Forsyth and Fulton use shapefiles sourced from the respective counties.

### 1. Load Primary Precinct Shapefile

In [ ]:
#Load in file
shapefiles_state = gp.read_file("./raw-from-source/Shapefiles/Georgia/vtd2020-shape-4-20-21/VTD2020-Shape.shp")




In [ ]:
print(shapefiles_state[shapefiles_state["CTYNAME"]=="COBB"]["PRECINCT_N"].unique())

#### Cobb: Split Bells Ferry 3/4, Dobbins 1/2, Marietta 3A/3B, Smyrna 3A/3B - Not Completed

#### Filter out Cobb, Gwinnett, DeKalb, Fulton and Forsyth

In [ ]:
#Make a copy of just Fulton (found that this was closer to VEST's final file)
filtered_fulton = shapefiles_state[shapefiles_state["FIPS2"]=="121"]

#Cobb, Gwinnett and DeKalb
"13135","13067","13089"

#Fulton and Forsyth
"13121","13117"

filter_list_old = ["135","067","089","121","117"]
filter_list = ["135","117","121"]

shapefiles_state = shapefiles_state[~shapefiles_state["FIPS2"].isin(filter_list)]

### 2. Load Census Redistricting Phase 2 Data

In [ ]:
#When downloading from the Census redistricing data program, counties are identified via a FIPS code
fips_codes_old = ["13135","13067","13089"]
fips_codes = ["13135"]

#Combine all the data from separate files into one
li = []
for i in fips_codes:
    ref = "./raw-from-source/Shapefiles/Census/partnership_shapefiles_19v2_"
    file_ref = ref+i+"/PVS_19_v2_vtd_"+i+".shp"
    file_prev = gp.read_file(file_ref)
    #print(file_prev.shape)
    li.append(file_prev)
shapefiles_census = pd.concat(li, axis=0, ignore_index=True)

In [ ]:
print(shapefiles_census)

### 3. Shapefile data from the counties themselves

In [ ]:
shapefiles_fulton = gp.read_file("./raw-from-source/Shapefiles/Fulton/Voting_Precincts/Voting_Precincts.shp")

shapefiles_forsyth  = gp.read_file("./raw-from-source/Shapefiles/Forsyth/Voting_Precinct-shp/Voting_Precinct.shp")

In [ ]:
shapefiles_forsyth.plot()

In [ ]:
print(vest_ga_20.crs)

shapefiles_forsyth = shapefiles_forsyth.to_crs(4019)
shapefiles_fulton = shapefiles_fulton.to_crs(4019)
shapefiles_census = shapefiles_census.to_crs(4019)

shapefiles_state.plot()

ax = shapefiles_state.plot()
shapefiles_forsyth.plot(ax = ax,color="red")
shapefiles_fulton.plot(ax = ax,color="orange")
shapefiles_census.plot(ax = ax,color="green")

In [ ]:
print(shapefiles_state.columns)
print(shapefiles_forsyth.columns)
print(shapefiles_fulton.columns)
print(shapefiles_census.columns)

In [ ]:
display(shapefiles_state.head())

In [ ]:
shapefiles_state = shapefiles_state[['DISTRICT', 'CTYSOSID', 'PRECINCT_I', 'PRECINCT_N', 'CNTY','CTYNAME','FIPS2','geometry']]
filtered_fulton = filtered_fulton[['DISTRICT', 'CTYSOSID', 'PRECINCT_I', 'PRECINCT_N', 'CNTY','CTYNAME','FIPS2','geometry']]
shapefiles_forsyth = shapefiles_forsyth[["PRECINCTID","NAME","COUNTY","geometry"]]
shapefiles_fulton = shapefiles_fulton[["OBJECTID","VoterDist","geometry"]]
shapefiles_census = shapefiles_census[["STATEFP","COUNTYFP","VTDST","NAMELSAD","NAME","geometry"]]

In [ ]:
display(vest_ga_20.columns)

shapefile_col_list = ['DISTRICT', 'CTYSOSID', 'PRECINCT_I', 'PRECINCT_N', 'CTYNAME','CTYNUMBER2', 'FIPS2','geometry']

In [ ]:
shapefiles_state.loc[:,"CTYNUMBER2"] = shapefiles_state.loc[:,"CNTY"]

shapefiles_state = shapefiles_state[shapefile_col_list]

In [ ]:
print(shapefiles_forsyth.head())

shapefiles_forsyth.loc[:,"DISTRICT"] = "117"+ shapefiles_forsyth.loc[:,"PRECINCTID"]
shapefiles_forsyth.loc[:,"CTYSOSID"] = shapefiles_forsyth.loc[:,"DISTRICT"]
shapefiles_forsyth.loc[:,"PRECINCT_I"] = shapefiles_forsyth.loc[:,"PRECINCTID"]
shapefiles_forsyth.loc[:,"PRECINCT_N"] = shapefiles_forsyth.loc[:,"PRECINCTID"] + " " + shapefiles_forsyth.loc[:,"NAME"]
shapefiles_forsyth.loc[:,"CTYNAME"] = "FORSYTH"
shapefiles_forsyth.loc[:,"CTYNUMBER2"] = "058"
shapefiles_forsyth.loc[:,"FIPS2"] = "117"

shapefiles_forsyth = shapefiles_forsyth[shapefile_col_list]

In [ ]:
shapefiles_fulton.head()

shapefiles_fulton.loc[:,"DISTRICT"] = "121"+ shapefiles_fulton.loc[:,"VoterDist"]
shapefiles_fulton.loc[:,"CTYSOSID"] = shapefiles_fulton.loc[:,"DISTRICT"]
shapefiles_fulton.loc[:,"PRECINCT_I"] = shapefiles_fulton.loc[:,"VoterDist"]
shapefiles_fulton.loc[:,"PRECINCT_N"] = shapefiles_fulton.loc[:,"VoterDist"]
shapefiles_fulton.loc[:,"CTYNAME"] = "FULTON"
shapefiles_fulton.loc[:,"CTYNUMBER2"] = "060"
shapefiles_fulton.loc[:,"FIPS2"] = "121"

shapefiles_fulton = shapefiles_fulton[shapefile_col_list]

In [ ]:
name_map_dict = {"135":"GWINNETT","067":"COBB","089":"DEKALB"}
num_map_dict = {"135":"067","067":"033","089":"044"}


shapefiles_census.loc[:,"PRECINCT_I"] = shapefiles_census.loc[:,"VTDST"].str[3:]
shapefiles_census.loc[:,"DISTRICT"] = shapefiles_census.loc[:,"COUNTYFP"]+ shapefiles_census.loc[:,"PRECINCT_I"]
shapefiles_census.loc[:,"CTYSOSID"] = shapefiles_census.loc[:,"DISTRICT"]

shapefiles_census.loc[:,"PRECINCT_N"] = shapefiles_census.loc[:,"NAME"]
shapefiles_census.loc[:,"CTYNAME"] = shapefiles_census.loc[:,"COUNTYFP"].map(name_map_dict)
shapefiles_census.loc[:,"CTYNUMBER2"] = shapefiles_census.loc[:,"COUNTYFP"].map(num_map_dict)
shapefiles_census.loc[:,"FIPS2"] = shapefiles_census.loc[:,"COUNTYFP"]

shapefiles_census = shapefiles_census[shapefile_col_list]

In [ ]:
ga_shapefile = pd.concat([shapefiles_census,shapefiles_state,shapefiles_fulton,shapefiles_forsyth])
ga_shapefile.reset_index(inplace=True,drop=True)

In [ ]:
print(ga_shapefile[ga_shapefile["CTYNAME"]=="COBB"]["PRECINCT_N"].unique())

In [ ]:
ga_shapefile["PRECINCT_N"] =ga_shapefile["PRECINCT_N"].str.upper()
ga_shapefile["unique_ID"] = ga_shapefile["FIPS2"]+"-"+ga_shapefile["PRECINCT_N"]

In [ ]:
ser = ga_shapefile["unique_ID"].value_counts(dropna=False)
ser[ser >1]

In [ ]:
ga_shapefile.loc[ga_shapefile["CTYSOSID"]=="0513-15C","unique_ID"] = "051-ELI WHITNEY COMPLEX 3-15C"
ga_shapefile.loc[ga_shapefile["CTYSOSID"]=="0512-06C","unique_ID"] = "051-ELI WHITNEY COMPLEX 2-06C"

ga_shapefile.loc[ga_shapefile["CTYSOSID"]=="0517-16C","unique_ID"] = "051-POOLER RECREATION CENTER GYMNASIUM 7-16C"
ga_shapefile.loc[ga_shapefile["CTYSOSID"]=="0517-12C","unique_ID"] = "051-POOLER RECREATION CENTER GYMNASIUM 7-12C"

In [ ]:
ser = ga_shapefile["unique_ID"].value_counts(dropna=False)
ser[ser >1]

In [ ]:
na_dist_list = ga_shapefile.loc[ga_shapefile['unique_ID'].isna()]["DISTRICT"]
print(ga_shapefile.loc[ga_shapefile['unique_ID'].isna()]["DISTRICT"].str[0:3].value_counts())
print(ga_shapefile[ga_shapefile["DISTRICT"].isin(na_dist_list)]["DISTRICT"].str[0:3].value_counts())

ga_shapefile = ga_shapefile[~((ga_shapefile['unique_ID'].isna())& (ga_shapefile["DISTRICT"].str[0:3]=="121"))]



print(ga_shapefile.loc[ga_shapefile['unique_ID'].isna()]["DISTRICT"].str[0:3].value_counts())
print(ga_shapefile[ga_shapefile["DISTRICT"].isin(na_dist_list)]["DISTRICT"].str[0:3].value_counts())

In [ ]:
print(vest_ga_20[vest_ga_20["FIPS2"]=="121"].shape)
print(ga_shapefile[ga_shapefile["FIPS2"]=="121"].shape)

In [ ]:
print(ga_shapefile.loc[ga_shapefile['unique_ID'].isna()])
ga_shapefile.loc[ga_shapefile['unique_ID'].isna(), 'unique_ID'] = ga_shapefile.loc[:,'DISTRICT']

## Shapefile Modifications

### Documentation

Three of the four VTDs in Chattahoochee County are comprised of Fort Benning. However, the county only reports one polling location for all voters, including residents of Fort Benning that vote within the county. The four Chattahoochee County VTDs have therefore been merged in the shapefile.

The following additional modifications reflect changes made prior to the 2020 general election:

**Barrow:** Merge 2/15, 3/12, 4/14, 5/7, 6/10/13, 8/9, 11/16; Adjust new 2/13 boundary  
**Bartow:** Split Cassville/Hamilton Crossing  
**Candler:** Merge Candler/Metter as Jack Strickland Comm Center  
**Chatham:** Split 7-7/8-16, 7-12/7-16; Realign 7-06C/7-07C  
**Chatooga:** Split Cloudland/Teloga along ridgeline that marks boundary between them with the USGS Topographic Contour shapefile  
**Clayton:** Split Ellenswood 1/2, Jonesboro 1/17/19, Lovejoy 3/6/7, Morrow 3/11, 5/10, Oak 3/5   
**Cobb:** Split Bells Ferry 3/4, Dobbins 1/2, Marietta 3A/3B, Smyrna 3A/3B  
**Columbia:** Split Bessie Thomas/2nd Mt Moriah, Harlem Branch/Harlem Senior Ctr; Merge Blanchard Park/MTZ Col FD;  Align multiple precincts with county maps  
**Coweta:** Merge Arts Centre/Jefferson Parkway as Newnan Centre  
**Fulton:** Merge CP07A/CP07D, CH01/CH04B, SS29A/SS29B, UC031/UC035  
**DeKalb:** Split Clarkston/Clarkston Comm Ctr; Realign Decatur/Oakhurst; Align precincts with Atlanta, Brookhaven, Decatur, Tucker city limits   
**Gwinnett:** Adjust Baycreek F/G, Berkshire J/M, Cates D/F, Garners C/B, Lawrenceville G/N, Pinckneyville S/T, Rockbridge A/G  
**Lowndes:** Split Northgate Assembly/Trinity, Jaycee/Mt Calvary/Northside/VSU  
**Oconee:** Merge Annex/City Hall; Align City Hall with Watkinsville city limits  
**Paulding:** Reorganize 12 precincts into 19 precincts as redrawn in 2019  
**Randolph:** Merge Carnegie/Cuthbert-Courthouse, 4th District/Fountain Bridge/Shellman  
**Troup:** Split Mountville between Gardner Newman/Hogansville/Rosemont; Align multiple precincts with county maps  
**Towns:** Merge Macedonia/Tate City  

Note that the leading zeros in the Paulding County precinct IDs are included in some election reports and omitted in others. The shapefile includes the leading zeros consistent with the voter file.  

### Fort Benning - Completed

> Three of the four VTDs in Chattahoochee County are comprised of Fort Benning. However, the county only reports one polling location for all voters, including residents of Fort Benning that vote within the county. The four Chattahoochee County VTDs have therefore been merged in the shapefile.

#### Take a look at the sourcefile versus VEST

In [ ]:
ax = vest_ga_20[vest_ga_20["CTYNAME"]=="CHATTAHOOCHEE"].boundary.plot(color="red")
ga_shapefile[ga_shapefile["CTYNAME"]=="CHATTAHOOCHEE"].plot(ax = ax,color="yellow")
ga_shapefile[ga_shapefile["DISTRICT"]=="053FTBEN1"].plot(ax = ax,color="blue")
ga_shapefile[ga_shapefile["DISTRICT"]=="053FTBEN2"].plot(ax = ax,color="green")
ga_shapefile[ga_shapefile["DISTRICT"]=="053FTBEN3"].plot(ax = ax,color="orange")

#### Merge the four VTDS and plot the result

In [ ]:
index_fort_1 = ga_shapefile[ga_shapefile["DISTRICT"]=="053FTBEN1"].index.values[0]
index_fort_2 = ga_shapefile[ga_shapefile["DISTRICT"]=="053FTBEN2"].index.values[0]
index_fort_3 = ga_shapefile[ga_shapefile["DISTRICT"]=="053FTBEN3"].index.values[0]

prec2combine = [ga_shapefile.loc[ga_shapefile["PRECINCT_N"]=="ACTIVITY CENTER"],ga_shapefile[ga_shapefile["DISTRICT"]=="053FTBEN1"],ga_shapefile[ga_shapefile["DISTRICT"]=="053FTBEN2"],ga_shapefile[ga_shapefile["DISTRICT"]=="053FTBEN3"]]
new_fort = gp.GeoDataFrame(pd.concat(prec2combine, ignore_index=True), crs = ga_shapefile.crs)

ga_shapefile.loc[ga_shapefile["PRECINCT_N"]=="ACTIVITY CENTER","geometry"]=new_fort.unary_union
ga_shapefile.loc[ga_shapefile["PRECINCT_N"]=="ACTIVITY CENTER","unique_ID"]="053-ACTIVITY CENTER (Includes FTBEN 1-3)"
ga_shapefile.loc[ga_shapefile["unique_ID"]=="053-ACTIVITY CENTER (Includes FTBEN 1-3)"].plot()
ga_shapefile = ga_shapefile.drop([index_fort_1,index_fort_2,index_fort_3])
ga_shapefile.reset_index(drop=True, inplace=True)

### Barrow: Merge 2/15, 3/12, 4/14, 5/7, 6/10/13, 8/9, 11/16; Adjust new 2/13 boundary - Completed
All of the merges seem to have already been completed, VEST doesn't provide much detail on the 2/13 boundary adjustment, but it's reasonable to assume that that's also been performed, as the others have been. Any differences will be plotted in the shapefile validation.

In [ ]:
print(ga_shapefile[ga_shapefile["CTYNAME"]=="BARROW"][["PRECINCT_N"]])
ax = ga_shapefile[(ga_shapefile["CTYNAME"]=="BARROW")&(ga_shapefile["PRECINCT_N"]=="02")].plot()
vest_ga_20[(vest_ga_20["CTYNAME"]=="BARROW")&(vest_ga_20["PRECINCT_N"]=="02 BETHLEHEM CHURCH - 211")].boundary.plot(ax = ax,color="red")

### Bartow: Split Cassville/Hamilton Crossing - Completed
This seems to have been performed already in the shapefile.

In [ ]:
ga_shapefile[(ga_shapefile["CTYNAME"]=="BARTOW") & (ga_shapefile["PRECINCT_N"].isin(["CASSVILLE","HAMILTON CROSSING"]))][["PRECINCT_N"]]

### Candler: Merge Candler/Metter as Jack Strickland Comm Center - Completed
This seems to have been performed already in the shapefile as this is the only precinct in Candler.

In [ ]:
ga_shapefile[ga_shapefile["CTYNAME"]=="CANDLER"][["PRECINCT_N"]]

### Chatham: Split 7-7/8-16, 7-12/7-16; Realign 7-06C/7-07C -  Completed
All of the splits and realignments already seem to have already been completed.

In [ ]:
ga_shapefile[(ga_shapefile["CTYNAME"]=="CHATHAM") & (ga_shapefile["PRECINCT_I"].str[0:4].isin(["7-07","8-16","7-12","7-16"]))]

In [ ]:
ax = ga_shapefile[(ga_shapefile["CTYNAME"]=="CHATHAM")&(ga_shapefile["PRECINCT_I"]=="7-07C")].plot()
vest_ga_20[(vest_ga_20["CTYNAME"]=="CHATHAM")&(vest_ga_20["PRECINCT_I"]=="7-07C")].boundary.plot(ax = ax,color="red")

### Chatooga: Split Cloudland/Teloga along ridgeline that marks boundary between them with the USGS Topographic Contour shapefile - Completed

In [ ]:
ga_shapefile[ga_shapefile["PRECINCT_N"]=="CLOUDLAND-TELOGA"].plot()

In [ ]:
#Printing with the merged precinct on top
ax2 = vest_ga_20[vest_ga_20["PRECINCT_N"]=="CLOUDLAND"].plot(color="blue")
vest_ga_20[vest_ga_20["PRECINCT_N"]=="TELOGA"].plot(ax=ax2,color="green")
ga_shapefile[ga_shapefile["PRECINCT_N"]=="CLOUDLAND-TELOGA"].boundary.plot(ax = ax2,color="red")

#Import the topo lines
elev_contour_1 = gp.read_file("./raw-from-source/Topographic_Lines/VECTOR_Dougherty_Gap_GA_7_5_Min_Shape/Elev_Contour.shp")
elev_contour_2 = gp.read_file("./raw-from-source/Topographic_Lines/VECTOR_Jamestown_AL_7_5_Min_Shape/Elev_Contour.shp")

#Plot the elev_countours
ax3 = elev_contour_1.plot(linewidth=0.1)
elev_contour_2.plot(ax=ax3,linewidth=0.1)
minx, miny, maxx, maxy = ga_shapefile[ga_shapefile["PRECINCT_N"]=="CLOUDLAND-TELOGA"].total_bounds
ax3.set_xlim(minx, maxx)
ax3.set_ylim(miny, maxy)

#Confirm that the precincts are really split on a ridgeline
ax4 = vest_ga_20[vest_ga_20["PRECINCT_N"]=="CLOUDLAND"].plot(color="blue",alpha=.5)
vest_ga_20[vest_ga_20["PRECINCT_N"]=="TELOGA"].plot(ax=ax4,color="green",alpha=.5)
elev_contour_1.plot(ax=ax4,linewidth=0.1)
elev_contour_2.plot(ax=ax4,linewidth=0.1)
minx, miny, maxx, maxy = ga_shapefile[ga_shapefile["PRECINCT_N"]=="CLOUDLAND-TELOGA"].total_bounds
ax4.set_xlim(minx, maxx)
ax4.set_ylim(miny, maxy)

#As you can see the precinct is not plotted exactly on 

In [ ]:
print(vest_ga_20.columns)

In [ ]:
#Merge the election results to check those
data_columns = ['G20PRERTRU', 'G20PREDBID',
       'G20PRELJOR', 'C20PRERTRU', 'C20PREDBID', 'C20PRELJOR', 'G20USSRPER',
       'G20USSDOSS', 'G20USSLHAZ', 'S20USSRLOE', 'S20USSRCOL', 'S20USSRGRA',
       'S20USSRJAC', 'S20USSRTAY', 'S20USSRJOH', 'S20USSDWAR', 'S20USSDJAC',
       'S20USSDLIE', 'S20USSDJOH', 'S20USSDJAM', 'S20USSDSLA', 'S20USSDWIN',
       'S20USSDTAR', 'S20USSLSLO', 'S20USSGFOR', 'S20USSIBUC', 'S20USSIBAR',
       'S20USSISTO', 'S20USSIGRE', 'G20PSCRSHA', 'G20PSCDBRY', 'G20PSCLMEL',
       'G20PSCRMCD', 'G20PSCDBLA', 'G20PSCLWIL']

for i in data_columns:
        ga_election.loc[ga_election["unique_ID"]=="055-CLOUDLAND",i]=int(ga_election.loc[ga_election["unique_ID"]=="055-CLOUDLAND",i])+int(ga_election.loc[ga_election["unique_ID"]=="055-TELOGA",i])

ga_election.loc[ga_election['precinct']=="CLOUDLAND",'unique_ID']="055-CLOUDLAND-TELOGA"       
ga_election = ga_election[ga_election["unique_ID"]!="055-TELOGA"]

In [ ]:
vest_ga_20[vest_ga_20["PRECINCT_N"]=="CLOUDLAND"].plot()
vest_ga_20[vest_ga_20["PRECINCT_N"]=="TELOGA"].plot()

prec2combine = [vest_ga_20.loc[vest_ga_20["PRECINCT_N"]=="CLOUDLAND"],vest_ga_20.loc[vest_ga_20["PRECINCT_N"]=="TELOGA"]]
new_prec = gp.GeoDataFrame(pd.concat(prec2combine, ignore_index=True), crs = vest_ga_20.crs)

vest_ga_20.loc[vest_ga_20["PRECINCT_N"]=="CLOUDLAND","geometry"]=new_prec.unary_union

vest_ga_20[vest_ga_20["PRECINCT_N"]=="CLOUDLAND"].plot()

for i in data_columns:
        vest_ga_20.loc[vest_ga_20["PRECINCT_N"]=="CLOUDLAND",i]=int(vest_ga_20.loc[vest_ga_20["PRECINCT_N"]=="CLOUDLAND",i])+int(vest_ga_20.loc[vest_ga_20["PRECINCT_N"]=="TELOGA",i])

vest_ga_20.loc[vest_ga_20["PRECINCT_N"]=="CLOUDLAND",'unique_ID']="055-CLOUDLAND-TELOGA"       
vest_ga_20 = vest_ga_20[vest_ga_20["PRECINCT_N"]!="TELOGA"]
vest_ga_20.reset_index(inplace=True,drop=True)


### Clayton: Split Ellenswood 1/2, Jonesboro 1/17/19, Lovejoy 3/6/7, Morrow 3/11, 5/10, Oak 3/5 - Completed
These changes appear to have been performed already

In [ ]:
precinct_list = ["ELLENWOOD 1","ELLENWOOD 2","JONESBORO 1","JONESBORO 17","JONESBORO 19",
                "LOVEJOY 3","LOVEJOY 6", "LOVEJOY 7", "MORROW 3","MORROW 11","MORROW 5", "MORROW 10", "OAK 3", "OAK 5"]

ga_shapefile[(ga_shapefile["CTYNAME"]=="CLAYTON")&(ga_shapefile["PRECINCT_N"].isin(precinct_list))][["PRECINCT_N"]]

### Cobb: Split Bells Ferry 3/4, Dobbins 1/2, Marietta 3A/3B, Smyrna 3A/3B - Completed

In [ ]:
precinct_list = ["BELLS FERRY 03","BELLS FERRY 04","DOBBINS 01","DOBBINS 02",
                "SMYRNA 3A","SMYRNA 3B", "MARIETTA 3A", "MARIETTA 3B"]

ga_shapefile[(ga_shapefile["CTYNAME"]=="COBB")&(ga_shapefile["PRECINCT_N"].isin(precinct_list))]

### Columbia: Split Bessie Thomas/2nd Mt Moriah, Harlem Branch/Harlem Senior Ctr; Merge Blanchard Park/MTZ Col FD;  Align multiple precincts with county maps - Completed

Any precincts that VEST re-aligned that weren't will print below

In [ ]:
prec_list = ["BESSIE THOMAS CENTER","SECOND MOUNT MORIAH BAPTIST CHURCH" ,"HARLEM SENIOR CENTER",
             "HARLEM BRANCH LIBRARY","BLANCHARD PARK","MTZ COL FIRE HDQTR."]
  

print(ga_shapefile[(ga_shapefile["CTYNAME"]=="COLUMBIA")&(ga_shapefile["PRECINCT_N"].isin(prec_list))]["PRECINCT_N"])

In [ ]:
ax = vest_ga_20[(vest_ga_20["CTYNAME"]=="COLUMBIA")&(vest_ga_20["PRECINCT_N"]=="BLANCHARD PARK")].plot()
ga_shapefile[(ga_shapefile["CTYNAME"]=="COLUMBIA")&(ga_shapefile["PRECINCT_N"]=="BLANCHARD PARK")].boundary.plot(ax = ax,color="red")

### Coweta: Merge Arts Centre/Jefferson Parkway as Newnan Centre - Completed

In [ ]:
ga_shapefile[ga_shapefile["PRECINCT_N"]=="THE NEWNAN CENTRE"]

In [ ]:
ga_shapefile[ga_shapefile["CTYNAME"]=="COWETA"]["PRECINCT_N"].unique()

### Fulton: Merge CP07A/CP07D, CH01/CH04B, SS29A/SS29B, UC031/UC035 - COMPLETED

In [ ]:
precinct_list = ["121CP07A","121CP07D","121CH01","121CH04B",
                "121SS29A","121SS29B", "121UC031", "121UC035"]

print(ga_shapefile[(ga_shapefile["DISTRICT"].isin(precinct_list))]["DISTRICT"])

In [ ]:
prec2combine = [ga_shapefile.loc[ga_shapefile["DISTRICT"]=="121CP07D"],ga_shapefile[ga_shapefile["DISTRICT"]=="121CP07A"]]
new_shp = gp.GeoDataFrame(pd.concat(prec2combine, ignore_index=True), crs = ga_shapefile.crs)
ga_shapefile.loc[ga_shapefile["DISTRICT"]=="121CP07D","geometry"]=new_shp.unary_union
ga_shapefile = ga_shapefile[ga_shapefile["DISTRICT"]!="121CP07A"]
ga_shapefile.reset_index(drop=True, inplace=True)

prec2combine = [ga_shapefile.loc[ga_shapefile["DISTRICT"]=="121CH04B"],ga_shapefile[ga_shapefile["DISTRICT"]=="121CH01"]]
new_shp = gp.GeoDataFrame(pd.concat(prec2combine, ignore_index=True), crs = ga_shapefile.crs).unary_union
ga_shapefile.loc[ga_shapefile["DISTRICT"]=="121CH01","geometry"]=gp.GeoDataFrame(geometry=[new_shp]).geometry.values
ga_shapefile = ga_shapefile[ga_shapefile["DISTRICT"]!="121CH04B"]
ga_shapefile.reset_index(drop=True, inplace=True)

prec2combine = [ga_shapefile.loc[ga_shapefile["DISTRICT"]=="121SS29B"],ga_shapefile[ga_shapefile["DISTRICT"]=="121SS29A"]]
new_shp = gp.GeoDataFrame(pd.concat(prec2combine, ignore_index=True), crs = ga_shapefile.crs)
ga_shapefile.loc[ga_shapefile["DISTRICT"]=="121SS29A","geometry"]=new_shp.unary_union
ga_shapefile = ga_shapefile[ga_shapefile["DISTRICT"]!="121SS29B"]
ga_shapefile.reset_index(drop=True, inplace=True)

prec2combine = [ga_shapefile.loc[ga_shapefile["DISTRICT"]=="121UC031"],ga_shapefile[ga_shapefile["DISTRICT"]=="121UC035"]]
new_shp = gp.GeoDataFrame(pd.concat(prec2combine, ignore_index=True), crs = ga_shapefile.crs)
ga_shapefile.loc[ga_shapefile["DISTRICT"]=="121UC031","geometry"]=new_shp.unary_union
ga_shapefile = ga_shapefile[ga_shapefile["DISTRICT"]!="121UC035"]
ga_shapefile.reset_index(drop=True, inplace=True)

In [ ]:
ax = ga_shapefile[(ga_shapefile["DISTRICT"]=="121CH01")].plot()
vest_ga_20[(vest_ga_20["DISTRICT"]=="121CH01")].boundary.plot(ax=ax,color="red")

ax = ga_shapefile[(ga_shapefile["DISTRICT"]=="121CP07D")].plot()
vest_ga_20[(vest_ga_20["DISTRICT"]=="121CP07D")].boundary.plot(ax=ax,color="red")

ax = ga_shapefile[(ga_shapefile["DISTRICT"]=="121SS29A")].plot()
vest_ga_20[(vest_ga_20["DISTRICT"]=="121SS29A")].boundary.plot(ax=ax,color="red")

ax = ga_shapefile[(ga_shapefile["DISTRICT"]=="121UC031")].plot()
vest_ga_20[(vest_ga_20["DISTRICT"]=="121UC031")].boundary.plot(ax=ax,color="red")

### DeKalb: Split Clarkston/Clarkston Comm Ctr; Realign Decatur/Oakhurst; Align precincts with Atlanta, Brookhaven, Decatur, Tucker city limits - Completed

In [ ]:
precinct_list = ['CLARKSTON COMMUNITY CENTER (CLA)','CLARKSTON (CLA)']
ga_shapefile[(ga_shapefile["CTYNAME"]=="DEKALB")&(ga_shapefile["PRECINCT_N"].isin(precinct_list))]["PRECINCT_N"]

In [ ]:
ax = ga_shapefile[(ga_shapefile["CTYNAME"]=="DEKALB")&(ga_shapefile["PRECINCT_N"]=="OAKHURST (DEC)")].plot()
ga_shapefile[(ga_shapefile["CTYNAME"]=="DEKALB")&(ga_shapefile["PRECINCT_N"]=='DECATUR (DEC)')].plot(ax=ax,color="green")

In [ ]:
ax = vest_ga_20[(vest_ga_20["CTYNAME"]=="DEKALB")&(vest_ga_20["PRECINCT_N"]=='OAKHURST')].plot()
vest_ga_20[(vest_ga_20["CTYNAME"]=="DEKALB")&(vest_ga_20["PRECINCT_N"]=='DECATUR')].plot(ax = ax,color="green")
 

#### Gwinnett: Adjust Baycreek F/G, Berkshire J/M, Cates D/F, Garners C/B, Lawrenceville G/N, Pinckneyville S/T, Rockbridge A/G
There isn't enough detail to check these changes.

### Lowndes: Split Northgate Assembly/Trinity, Jaycee/Mt Calvary/Northside/VSU - Completed

In [ ]:
ga_shapefile[ga_shapefile["CTYNAME"]=="LOWNDES"]["PRECINCT_N"].unique()

### Oconee: Merge Annex/City Hall; Align City Hall with Watkinsville city limits - Completed

In [ ]:
ga_shapefile[ga_shapefile["CTYNAME"]=="OCONEE"]["PRECINCT_N"].unique()

ax = ga_shapefile[ga_shapefile["PRECINCT_N"]=='CITY HALL'].plot()
vest_ga_20[vest_ga_20["PRECINCT_N"]=='CITY HALL'].boundary.plot(ax = ax,color="red")

### Paulding: Reorganize 12 precincts into 19 precincts as redrawn in 2019 - Completed

In [ ]:
ga_shapefile[ga_shapefile["CTYNAME"]=="PAULDING"].shape

### Randolph: Merge Carnegie/Cuthbert-Courthouse, 4th District/Fountain Bridge/Shellman - Completed

In [ ]:
ga_shapefile[ga_shapefile["CTYNAME"]=="RANDOLPH"]["PRECINCT_N"]

### Troup: Split Mountville between Gardner Newman/Hogansville/Rosemont; Align multiple precincts with county maps - Completed

In [ ]:
ga_shapefile[ga_shapefile["CTYNAME"]=="TROUP"]["PRECINCT_N"]

prec_list = ["GARDNER NEWMAN","HOGANSVILLE","ROSEMONT"]

ga_shapefile[(ga_shapefile["CTYNAME"]=="TROUP")&(ga_shapefile["PRECINCT_N"].isin(prec_list))].plot(column="PRECINCT_N")
vest_ga_20[(vest_ga_20["CTYNAME"]=="TROUP")&(vest_ga_20["PRECINCT_N"].isin(prec_list))].plot(column="PRECINCT_N")

### Towns: Merge Macedonia/Tate City - Completed

In [ ]:
ga_shapefile[ga_shapefile["CTYNAME"]=="TOWNS"]["PRECINCT_N"].unique()

### Attempt to Join

In [ ]:
join_attempt_3 = pd.merge(ga_election,ga_shapefile,how="outer",on="unique_ID",indicator=True)
print(join_attempt_3["_merge"].value_counts())

left_only = join_attempt_3[join_attempt_3["_merge"]=="left_only"]
right_only = join_attempt_3[join_attempt_3["_merge"]=="right_only"]

left_only.to_csv("./source_join_elections.csv")
right_only.to_csv("./source_join_shapes.csv")

In [ ]:
example_fips_dict ={'013-01': '013-01 BETHLEHEM COMMUNITY CENTER',
 '013-02': '013-02 BETHLEHEM CHURCH - 211',
 '013-03': '013-03 HMONG NEW HOPE ALLIANCE CHURCH',
 '013-04': '013-04 COVENANT LIFE SANCTUARY',
 '013-05': '013-05 FIRE STATION 1 (STATHAM)',
 '013-08': '013-08 FIRST BAPTIST CHURCH WINDER',
 '013-13': '013-13 WINDER COMMUNITY CENTER',
 '013-16': '013-16 THE CHURCH AT WINDER',
 '023-1': '023-FAIRGROUND',
 '029-J.F.GREGORY PARK': '029-JF GREGORY PARK',
 '029-PUBLIC SAFETY COMPX': '029-PUBLIC SAFETY COMPLEX',
 '029-RH RECREATION COMPX': '029-RH REC COMPLEX',
 '033-ST. CLAIR': '033-ST CLAIR',
 '039-ST. MARYS': '039-ST MARYS',
 '039-WEST SAINT MARYS': '039-WEST ST MARYS',
 '043-JACK STRICKLAND COMMUNITY CENTER': '043-JACK STRICKLAND COMM CENTER',
 '045-UNIVERSITY OF W. GA': '045-UNIVERSITY OF W GA',
 '049-ST. GEORGE': '049-ST GEORGE',
 '051-J E A': '051-JEA BUILDING',
 '051-ST. THOMAS EPISCOPAL CHURCH': '051-ST THOMAS EPISCOPAL CHURCH',
 '051-ISLANDS CHRISTIAN CH': '051-ISLANDS CHRISTIAN CHURCH',
 '051-FELLOWSHIP OF LOVE COMMUNITY CHURCH': '051-FELLOWSHIP OF LOVE CHURCH',
 '051-CHRIST COMMUNITY CHURCH AT MORNINGSIDE': '051-CHRIST COMMUNITY CHURCH',
 '051-COKESBURY METHODIST CHURCH': '051-COKESBURY METHODIST',
 '051-SOUTHSIDE BAPTIST CH': '051-SOUTHSIDE BAPTIST CHURCH',
 '051-THUNDERBOLT MUN COMP': '051-THUNDERBOLT MUNI COMPLEX',
 '051-FRANK MURRAY COMMUNITY CENTER': '051-FRANK MURRAY COMM CENTER',
 '051-SAINT FRANCIS EPISCOPAL CHURCH': '051-ST FRANCIS EPISCOPAL CHURCH',
 '051-FIRST BAPTIST CHURCH OF THE ISLAND': '051-FIRST BAPTIST OF THE ISLAND',
 '051-WILMINGTON ISLAND UNITED METHODIST CHURCH': '051-WILMINGTON ISLAND UMC',
 '051-WILMINGTON ISLAND PRESBYTERIAN CHURCH': '051-WILMINGTON ISLAND PRES CHURCH',
 '051-TYBEE IS SCH CAFE': '051-TYBEE ISLAND SCHOOL CAFE',
 '051-SKID IS PRES CHURCH': '051-SKIDAWAY ISLAND PRES CHURCH',
 '051-SKIDAWAY ISLAND STAT': '051-SKIDAWAY ISLAND STATE PARK',
 '051-SENIOR CITIZEN CENTER': '051-SENIOR CITIZENS CENTER',
 '051-BUTLER PRESBYTERIAN CHURCH ED BLDG': '051-BUTLER PRESBYTERIAN CHURCH',
 '051-LIBERTY CITY COMMUNITY CENTER': '051-LIBERTY CITY COMM CTR',
 '051-STATION 1': '051-ELKS LODGE',
 '051-SAVANNAH PRIMITIVE BAPTIST CHURCH': '051-SAVANNAH PRIMITIVE BC',
 '051-LARGO-TIBET ELEMENTA': '051-LARGO-TIBET ELEMENTARY',
 '051-WHITE BLUFF PRESBYTERIAN CHURCH': '051-WHITE BLUFF PRESBYTERIAN',
 '051-WINDSOR FOREST BAPTIST CHURCH SCHOOL': '051-WINDSOR FOREST BAPTIST',
 '051-CRUSADER COMMUNITY CENTER': '051-CRUSADER COMM CENTER',
 '051-STATION 3': '051-GEORGETOWN ELEMENTARY',
 '051-GARDEN CITY SENIOR CENTER': '051-GARDEN CITY SENIOR CTR',
 '051-PB EDWARDS GYMNASIUM': '051-PB EDWARDS GYM',
 '051-LAKE SHORE COMMUNITY CENTER': '051-LAKE SHORE COMM CTR',
 '051-BLOOMINGDALE COMMUNITY CENTER': '051-BLOOMINGDALE COMM CTR',
 '051-SAVANNAH HOLY CHURCH OF GOD': '051-SAVANNAH HOLY C OF G',
 '051-PROGRESSIVE REC CTR': '051-PROGRESSIVE REC CENTER',
 '051-COURTYARD BY MARRIOTT': '051-SEVENTH DAY ADV CHURCH',
 '051-SOUTHSIDE FIRE TRAINING CENTER': '051-SOUTHSIDE FIRE TRNG CTR',
 '051-WEST BROAD STREET YMCA': '051-W BROAD ST YMCA',
 '051-TOMPKINS RECREATION CENTER': '051-TOMPKINS REC CENTER',
 '051-WOODVILLE-TOMPKINS TECHNICAL INST': '051-WOODVILLE-TOMPKINS TI',
 '051-RESURRECTION OF OUR LORD CATHOLIC CHURCH': '051-RESURRECTION OF OUR LORD CHURCH',
 '051-CARVER HEIGHTS COMM': '051-CARVER HEIGHTS COMM CTR',
 '051-BUTLER ELEMENTARY SC': '051-BUTLER ELEMENTARY',
 '051-SAVANNAH CHRISTIAN S': '051-SAVANNAH CHRISTIAN',
 '051-FIRST BAPTIST CH GARDEN CITY': '051-GARDEN CITY REC CENTER',
 '051-MIGHTY EIGHTH MUSEUM': '051-ROYAL CINEMAS AND IMAX',
 '051-POOLER RECREATION CENTER GYMNASIUM 7-12C':'051-POOLER CHURCH',
 '051-POOLER RECREATION CENTER GYMNASIUM 7-16C':'051-POOLER REC CENTER GYM',
 '053-ACTIVITY CENTER': '053-ACTIVITY CENTER (Includes FTBEN 1-3)',
 '063-OAK5': '063-OAK 5',
 '067-MCEACHERN': '067-MCEACHERN 01',
 '067-MOUNT BETHEL 01': '067-MT BETHEL 01',
 '067-MOUNT BETHEL 03': '067-MT BETHEL 03',
 '067-MOUNT BETHEL 04': '067-MT BETHEL 04',
 '073-BLUERIDGE ELEMENTARY': '073-BLUE RIDGE ELEMENTARY',
 '073-CHRIST CHURCH, PRESBYTERIAN': '073-CHRIST CHURCH PRESBYTERIAN',
 '073-CHRIST THE KING CH': '073-CHRIST THE KING LUTH CHURCH',
 '073-COL CTY BOARD OF EDU': '073-COL CTY BD OF EDU',
 '073-DAMASCUS BAPT CHURCH': '073-DAMASCUS BAPTIST CHURCH',
 '073-EUBANK/BLANCHARD CTR': '073-EUBANK-BLANCHARD CENTER',
 '073-GROVETOWN METHODIST': '073-GROVETOWN METHODIST CHURCH',
 '073-GROVETOWN DEPT OF PUBLIC SAFETY STATION #2': '073-GROVETOWN PUBLIC SAFETY STATION 2',
 '073-JOURNEY COMM. CHURCH': '073-JOURNEY COMM CHURCH',
 '073-KIOKEE BAPT CHURCH': '073-KIOKEE BAPTIST CHURCH',
 '073-LEWIS METHODIST': '073-LEWIS METHODIST CHURCH',
 '073-LIBERTY PARK - GROVETOWN': '073-LIBERTY PARK-GROVETOWN',
 '073-MTZ COL FIRE HDQTR.': '073-MTZ COL FIRE HDQTR',
 '073-SECOND MT. CARMEL BAPTIST CHURCH': '073-SECOND MT CARMEL BAPTIST CHURCH',
 '073-SECOND MOUNT MORIAH BAPTIST CHURCH': '073-SECOND MT MORIAH BAPTIST CHURCH',
 '073-TRINITY BAPT CHURCH': '073-TRINITY BAPTIST CHURCH',
 '073-WESTSIDE BAPT CHURCH': '073-WESTSIDE BAPTIST CHURCH',
 '073-WOODLAWN BAPT CHURCH': '073-WOODLAWN BAPTIST CHURCH',
 '075-NEW LIFE BAPTIST CH.': '075-NEW LIFE BAPTIST CHURCH',
 '077-THE NEWNAN CENTRE': '077-NEWNAN CENTRE',
 '087-MT. PLEASANT': '087-MT PLEASANT',
 '089-ASHFORD DUNWOODY RD (BHAVN)': '089-ASHFORD DUNWOODY RD',
 '089-ASHFORD PARK ELEM (BHAVN)': '089-ASHFORD PARK ELEM',
 '089-ASHFORD PARKSIDE (BHAVN)': '089-ASHFORD PARKSIDE',
 '089-AUSTIN (DUN)': '089-AUSTIN',
 '089-AVONDALE (AVO)': '089-AVONDALE',
 '089-BOULEVARD (ATL)': '089-BOULEVARD',
 '089-BRIAR VISTA ELEM (UNI & ATL)': '089-BRIAR VISTA ELEM',
 '089-BRIARWOOD (BHAVN)': '089-BRIARWOOD',
 '089-BROCKETT (TUC)': '089-BROCKETT',
 '089-BROCKETT ELEM (TUC)': '089-BROCKETT ELEM',
 '089-BROOKHAVEN (BHAVN)': '089-BROOKHAVEN',
 '089-BURGESS ELEM (ATL)': '089-BURGESS ELEM',
 '089-CHAMBLEE (CHA)': '089-CHAMBLEE',
 '089-CHAMBLEE 2 (CHA)': '089-CHAMBLEE 2',
 '089-CHESNUT ELEM (DUN)': '089-CHESNUT ELEM',
 '089-CLAIREMONT EAST (DEC)': '089-CLAIREMONT EAST',
 '089-CLAIREMONT WEST (DEC)': '089-CLAIREMONT WEST',
 '089-COAN RECREATION CENTER (ATL)': '089-COAN RECREATION CENTER',
 '089-CROSS KEYS HIGH (BHAVN)': '089-CROSS KEYS HIGH',
 '089-DORAVILLE NORTH (DOR)': '089-DORAVILLE NORTH',
 '089-DORAVILLE SOUTH (DOR)': '089-DORAVILLE SOUTH',
 '089-DRESDEN ELEM (CHA)': '089-DRESDEN ELEM',
 '089-DRUID HILLS': '089-DRUID HILLS HIGH',
 '089-DUNWOODY (DUN)': '089-DUNWOODY',
 '089-DUNWOODY 2 (DUN)': '089-DUNWOODY 2',
 '089-DUNWOODY LIBRARY (DUN)': '089-DUNWOODY LIBRARY',
 '089-EAST LAKE (ATL)': '089-EAST LAKE',
 '089-EMORY SOUTH (UNI & ATL)': '089-EMORY SOUTH',
 '089-FERNBANK': '089-FERNBANK ELEM',
 '089-GEORGETOWN (DUN)': '089-GEORGETOWN SQ',
 '089-GLENNWOOD (DEC)': '089-GLENNWOOD',
 '089-HARRIS - MARGARET HARRIS ED': '089-HARRIS-MARGARET HARRIS ED',
 '089-HARRIS - NARVIE J. HARRIS ELEM': '089-HARRIS-NARVIE J HARRIS ELEM',
 '089-HUGH HOWELL (TUC)': '089-HUGH HOWELL',
 '089-HUNTLEY HILLS ELEM (CHA)': '089-HUNTLEY HILLS ELEM',
 '089-IDLEWOOD ELEM (TUC)': '089-IDLEWOOD ELEM',
 '089-JOHNSON ESTATES (ATL)': '089-JOHNSON ESTATES',
 '089-KINGSLEY ELEM (DUN)': '089-KINGSLEY ELEM',
 '089-KITTREDGE ELEM (BHAVN)': '089-KITTREDGE ELEM',
 '089-KNOLLWOOD ELEM': '089-KNOLLWOOD',
 '089-LIN - MARY LIN ELEM (ATL)': '089-LIN-MARY LIN ELEM',
 '089-LITHONIA (LIT)': '089-LITHONIA',
 '089-MATHIS - BOB MATHIS ELEM': '089-MATHIS-BOB MATHIS ELEM',
 '089-METROPOLITAN (ATL)': '089-METROPOLITAN',
 '089-MIDVALE ELEM (TUC)': '089-MIDVALE ELEM',
 '089-MIDWAY ELEM': '089-MIDWAY',
 '089-MILLER - ELDRIDGE L. MILLER ELEM': '089-MILLER-ELDRIDGE L MILLER ELEM',
 '089-MONTCLAIR ELEM (BHAVN)': '089-MONTCLAIR ELEM',
 '089-MONTGOMERY ELEM (BHAVN)': '089-MONTGOMERY ELEM',
 '089-MONTREAL (TUC)': '089-MONTREAL',
 '089-MOUNT VERNON EAST (DUN)': '089-MOUNT VERNON EAST',
 '089-MT.VERNON WEST (DUN)': '089-MOUNT VERNON WEST',
 '089-NORTH PEACHTREE (DUN)': '089-NORTH PEACHTREE',
 '089-OAKHURST (DEC)': '089-OAKHURST',
 '089-PEACHTREE MIDDLE (DUN)': '089-PEACHTREE MIDDLE',
 '089-PINE LAKE (PIN)': '089-PINE LAKE',
 '089-PONCE DE LEON (DEC)': '089-PONCE DE LEON',
 '089-SHAW ELEMENTARY': '089-SHAW-ROBERT SHAW ELEM',
 '089-SILVER LAKE (ATL & BHAVN)': '089-SILVER LAKE',
 '089-SKYLAND (BHAVN)': '089-SKYLAND',
 '089-SMOKE RISE (TUC)': '089-SMOKE RISE',
 '089-SNAPFINGER ROAD NORTH': '089-SNAPFINGER ROAD N',
 '089-SNAPFINGER ROAD SOUTH': '089-SNAPFINGER ROAD S',
 '089-STONE MOUNTAIN (STO)': '089-STONE MTN',
 '089-STONE MOUNTAIN CHAMPION (STO)': '089-STONE MTN CHAMPION',
 '089-STONE MOUNTAIN MIDDLE (TUC)': '089-STONE MTN MIDDLE',
 '089-TILLY MILL ROAD (DUN)': '089-TILLY MILL ROAD',
 '089-TUCKER (TUC)': '089-TUCKER',
 '089-TUCKER LIBRARY (TUC)': '089-TUCKER LIBRARY',
 '089-WINNONA PARK (DEC)': '089-WINNONA PARK',
 '089-WINTERS CHAPEL (DUN)': '089-WINTERS CHAPEL',
 '089-WOODWARD ELEM (BHAVN)': '089-WOODWARD',
'089-CANDLER PARK (ATL)':'089-CANDLER PARK',
'089-CLARKSTON (CLA)':'089-CLARKSTON',
'089-CLARKSTON COMMUNITY CENTER (CLA)':'089-CLARKSTON COMMUNITY CENTER',
'089-DECATUR (DEC)':'089-DECATUR',
'089-GEORGETOWN SQ (DUN)':'089-GEORGETOWN SQ',
'089-LIN-MARY LIN ELEM (ATL)':'089-LIN-MARY LIN ELEM',
'089-MONTGOMERY ELEM  (BHAVN)':'089-MONTGOMERY ELEM',
'089-MOUNT VERNON WEST (DUN)':'089-MOUNT VERNON WEST',
'089-NARVIE J HARRIS ELEM':'089-HARRIS-NARVIE J HARRIS ELEM',
'089-WOODWARD (BHAVN)':'089-WOODWARD',
 '091-CHAUN': '091-CHAUNCEY',
 '091-EMPIR': '091-EMPIRE',
 '091-JAYBI': '091-JAYBIRD',
 '091-MCCRA': '091-MCCRANIE',
 '091-MITCH': '091-MITCHELL',
 '091-MULLI': '091-MULLIS',
 '091-PLAIN': '091-PLAINFIELD',
 '091-PONDT': '091-PONDTOWN',
 '091-RAWLI': '091-RAWLINS',
 '091-VILUL': '091-VILULA',
 '091-YONKE': '091-YONKER',
 '095-ALICE COACHMAN ELEM': '095-ALICE COACHMAN ELEMENTARY',
 '095-INTERNATIONAL STUDIE': '095-INTERNATIONAL STUDIES',
 '095-JACKSON HEIGHTS ELEM': '095-JACKSON HEIGHTS ELEMENTARY',
 '095-LAMAR REESE ELEM SCH': '095-LAMAR REESE ELEMENTARY',
 '095-PALMYRA METH CHURCH': '095-PALMYRA METHODIST',
 '095-PINE BLUFF BAPT CHUR': '095-PINE BLUFF BAPTIST CHURCH',
 '095-PUTNEY 1ST BAPT CHUR': '095-PUTNEY 1ST BAPTIST CHURCH',
 '095-SHERWOOD ELEM SCHOOL': '095-SHERWOOD ELEMENTARY',
 '095-SHILOH BAPTIST CHURC': '095-SHILOH BAPTIST CHURCH',
 '095-TURNER ELEM SCHOOL': '095-TURNER ELEMENTARY',
 '095-WESTTOWN ELEM SCHOOL': '095-WESTTOWN ELEMENTARY',
 '097-EPHESUS BAPTIST CHUR': '097-EPHESUS BAPTIST CHURCH',
 '097-GOLDEN METHODIST CH': '097-GOLDEN METHODIST CHURCH',
 '097-LUTHERAN CHURCH - GS': '097-LUTHERAN CHURCH-GS',
 '097-MIRROR LAKE ELEMENTA': '097-MIRROR LAKE ELEMENTARY',
 '109-VETERANS COMM CTR': '109-VETERANS COMMUNITY CENTER',
 "115-FOSTER'S MILL": '115-FOSTERS MILL',
 '127-BLYTHE ISLAND BAPTIST CHURCH': '127-BLYTHE ISLAND BAPTIST',
 '127-COLLEGE PLACE': '127-COLLEGE PLACE UMC',
 '127-NORTHSIDE CHURCH': '127-NORTHSIDE BAPTIST',
 '127-ST WILLIAM CHURCH': '127-ST WILLIAM CATHOLIC CHURCH',
 '127-STERLING CH. OF GOD': '127-STERLING CHURCH OF GOD',
 '131-CAIRO 4TH DISTR': '131-CAIRO 4TH DISTRICT',
 '131-CAIRO 5TH DISTR': '131-CAIRO 5TH DISTRICT',
 '135-PINKCNEYVILLE A': '135-PINCKNEYVILLE A',
 '137-TOWN OF MOUNT AIRY': '137-TOWN OF MT AIRY',
 '141-ST. MARK': '141-ST MARK',
 '149-COOKSVILLE - CORINTH': '149-COOKSVILLE-CORINTH',
 '151-MT. BETHEL': '151-MT BETHEL',
 '151-MOUNT CARMEL': '151-MT CARMEL',
 '159-MARTIN - BURNEY': '159-MARTIN-BURNEY',
 '171-CHAPPELL MILL V. FD': '171-CHAPPELL MILL VFD',
 '175-FBC - FLC': '175-FBC-FLC',
 '175-FIRE DEPT STA #5': '175-FIRE DEPT STA 5',
 '175-LCFS #10 (VALAMBROSIA)': '175-LCFS 10 VALAMBROSIA',
 '175-RURAL FIRE STA #17': '175-RURAL FIRE STA 17',
 '175-W T ADAMS FIRE STA #18': '175-W T ADAMS FIRE STA 18',
 '177-#3 CJC': '177-CJC',
 '177-#9 CENTURY FIRE STAT': '177-CENTURY FIRE STATION',
 '177-#1 CHOKEE': '177-CHOKEE',
 '177-#6 FIRST BAPTIST': '177-FIRST BAPTIST',
 '177-#8 FLINT REFORMED BAPTIST': '177-FLINT REFORMED BAPTIST',
 '177-#5 FRIENDSHIP BAPT.': '177-FRIENDSHIP BAPTIST',
 '177-#4 LEESBURG': '177-LEESBURG',
 '177-#10 REDBONE': '177-REDBONE',
 '177-#7 SDA CHURCH': '177-SDA CHURCH',
 '177-#2 SMITHVILLE': '177-SMITHVILLE',
 '179-MEMORIAL DR. EAST': '179-MEMORIAL DR EAST',
 '181-FAITH TEMPLE OF LINC': '181-FAITH TEMPLE OF LINCOLN',
 '199-UPPER 9TH -ALVATON': '199-ALVATON',
 '199-THIRD-DURAND': '199-DURAND',
 '199-MIDDLE 9TH-GAY': '199-GAY',
 '199-8TH-GREENVILLE': '199-GREENVILLE',
 '199-7TH-ODESSADALE': '199-ODESSADALE',
 '199-2ND-WARM SPRINGS': '199-WARM SPRINGS',
 '199-LOWER 9TH-WOODBURY': '199-WOODBURY',
 '211-3. BETH/SPRINGFIELD': '211-BETH/SPRINGFIELD',
 '211-4. CENTRAL MORGAN': '211-CENTRAL MORGAN',
 '211-5. CLACKS CHAPEL': '211-CLACKS CHAPEL',
 '211-2. EAST MORGAN': '211-EAST MORGAN',
 '211-7. NORTH MORGAN': '211-NORTH MORGAN',
 '211-1. NORTHEAST MORGAN': '211-NORTHEAST MORGAN',
 '211-6. WEST MORGAN': '211-WEST MORGAN',
 '215-GENTIAN/REESE @LDS': '215-GENTIAN/REESE',
 '215-ST. PETER': '215-ST PETER',
 '223-DIANNE WRIGHT INNOVATION CENTER': '223-D WRIGHT INNOVATION CTR',
 '223-THE EVENTS PLACE': '223-EVENTS PLACE',
 '223-LEGACY BAPTIST CHURCH': '223-LEGACY BAPT CHURCH',
 '223-NEBO ELEMENTARY SCHOOL': '223-NEBO ELEM SCHOOL',
 '223-PAULDING SENIOR CENTER': '223-PAULDING SR CENTER',
 '223-POPLAR SPRINGS BAPTIST CHURCH': '223-POPLAR SPRGS BAPT CHURCH',
 '223-RUSSOM ELEMENTARY SCHOOL': '223-RUSSOM ELEMENTARY',
 '223-SHELTON ELEMENTARY SCHOOL': '223-SHELTON ELEMENTARY',
 '223-TAYLOR FARM PARK MEETING ROOM': '223-TAYLOR FARM PARK',
 '223-WATSON GOVT COMPLEX': '223-WATSON GOVT CMPLX',
 '225-BYRON #1': '225-BYRON 1',
 '225-BYRON #2': '225-BYRON 2',
 '225-FORT VALLEY #1': '225-FORT VALLEY 1',
 '225-FORT VALLEY #2': '225-FORT VALLEY 2',
 '225-FORT VALLEY #3': '225-FORT VALLEY 3',
 '229-HACKLEBARNEY/CASON': '229-HACKLEBARNEY-CASON',
 '229-ST JOHNS/BLACKSHEAR': '229-ST JOHNS-BLACKSHEAR',
 '229-SUNSET/SWEAT': '229-SUNSET-SWEAT',
 '243-CUTHBERT/COURTHOUSE': '243-CUTHBERT-COURTHOUSE',
 '251-SCREVEN REC. DEPT.': '251-SCREVEN REC DEPT',
 '261-AG CENTER': '261-AGRI-CENTER',
 '261-REES PARK': '261-REESE PARK',
 '261-THOMSON': '261-THOMPSON',
 '275-TC PUBLIC LIBRARY/JERGER': '275-JERGER',
 '275-LITTLE OCHLOCKNEE BAPTIST': '275-LITTLE OCHLOCKNEE',
 '275-MERRILVILLE': '275-MERRILLVILLE',
 '275-TC PUBLIC LIBRARY/SCOTT': '275-SCOTT',
 '279-513 V.P.D.': '279-513 VPD',
 '279-514 S.T.I.A.L.C.': '279-514 STIALC',
 '285-MCLENDON': '285-MCCLENDON',
 '299-100': '299-District 1',
 '299-200A': '299-District 2A',
 '299-200B': '299-District 2B',
 '315-ABBEVILLE NORTH #2': '315-ABBEVILLE NORTH 2',
 '315-ABBEVILLE SOUTH #5': '315-ABBEVILLE SOUTH 5',
 '315-PINEVIEW #2': '315-PINEVIEW 2',
 '315-PITTS #3': '315-PITTS 3',
 '315-ROCHELLE NORTH #1': '315-ROCHELLE NORTH 1',
 '315-ROCHELLE SOUTH #4': '315-ROCHELLE SOUTH 4',
 '317-SENIOR CITIZEN CENTE': '317-SENIOR CITIZEN CENTER',
 '089-CANDLER - MURPHEY CANDLER ELEM': '089-CANDLER-MURPHEY CANDLER ELEM',
 #The below changes are based on precinct numbers
 '299-300': '299-1231-150B',
 '299-EMERSON PARK BAPTIST CHURCH': '299-1231-150C',
 '299-400': '299-1231-151',
 '299-404': '299-Beach-Bickley',
 '299-405': '299-Haywood',
 '299-406': '299-Jamestown',
 '299-407': '299-Manor',
 '299-408': '299-Millwood',
 '299-409': '299-Waresboro',
 #The below gives justification for this seemingly arbitrary name changes
  #https://www.dekalbcountyga.gov/sites/default/files/users/user305/NOTICE%20OF%20CHANGE%20OF%20POLLING%20PLACES.pdf
 '089-KING - ML KING JR HIGH': '089-SNAPFINGER ROAD',
 '089-EPWORTH (ATL)': '089-CANDLER PARK'
}

In [ ]:
ga_shapefile["unique_ID"] = ga_shapefile["unique_ID"].map(example_fips_dict).fillna(ga_shapefile["unique_ID"])

In [ ]:
join_attempt_3 = pd.merge(ga_election,ga_shapefile,how="outer",on="unique_ID",indicator=True)
print(join_attempt_3["_merge"].value_counts())

left_only = join_attempt_3[join_attempt_3["_merge"]=="left_only"]
right_only = join_attempt_3[join_attempt_3["_merge"]=="right_only"]

ga_source = join_attempt_3[join_attempt_3["_merge"]=="both"]

left_only.to_csv("./source_join_elections.csv")
right_only.to_csv("./source_join_shapes.csv")

In [ ]:
final_merge = pd.merge(ga_source,vest_ga_20,how="outer",on="unique_ID",indicator="final_merge")

print(final_merge["final_merge"].value_counts())

In [ ]:
both = final_merge[final_merge["final_merge"]=="both"]
vest_geoms = gp.GeoDataFrame(both,geometry="geometry_x",crs=vest_ga_20.crs)
source_geoms = gp.GeoDataFrame(both,geometry="geometry_y",crs=vest_ga_20.crs)
source_geoms = source_geoms.to_crs(3857)
vest_geoms = vest_geoms.to_crs(3857)
source_geoms["geometry_x"]=source_geoms.buffer(0)
vest_geoms["geometry_y"]=vest_geoms.buffer(0)
vals = source_geoms.geom_almost_equals(vest_geoms,decimal=0)
#print(vals.value_counts())

In [ ]:
#Get the area

In [ ]:
ga_shapefile[ga_shapefile["unique_ID"]=="089-GLENNWOOD"].area

In [ ]:
vest_ga_20[vest_ga_20["unique_ID"]=="089-GLENNWOOD"].plot()

In [ ]:
count = 0
area_list = []
for i in range(0,len(source_geoms)):
    try:
        diff = source_geoms.iloc[[i]].symmetric_difference(vest_geoms.iloc[[i]])
        intersection = source_geoms.iloc[[i]].intersection(vest_geoms.iloc[[i]])
        area = float(diff.area/10e6)
        area_list.append(area)
            #print("Area is " + str(area))

        if (area > .5):
            count += 1
            name = source_geoms.iat[i,0]
            print(str(count)+") For " + name + " difference in area is " + str(area))
            if (intersection.iloc[0].is_empty):
                base = diff.plot(color="red")
                source_geoms.iloc[[i]].plot(color="orange",ax=base)
                vest_geoms.iloc[[i]].plot(color="blue",ax=base)
                base.set_title(name)
            else:
                base = diff.plot(color="red")
                source_geoms.iloc[[i]].plot(color="orange",ax=base)
                vest_geoms.iloc[[i]].plot(color="blue",ax=base)
                intersection.plot(color="green",ax=base)
                base.set_title(name)
    except:
        print("Not able to run code for: ", source_geoms.iat[i,0])
        #diff = source_geoms.iloc[[i]].symmetric_difference(vest_geoms.iloc[[i]])


In [ ]:
df = pd.DataFrame(area_list)
print(df.shape)

print(str(len(df[df[0]==0]))+" precincts w/ a difference of 0 km^2")
print(str(len(df[(df[0]<.1) & (df[0]>0)]))+ " precincts w/ a difference between 0 and .1 km^2")
print(str(len(df[(df[0]<.5) & (df[0]>=.1)]))+ " precincts w/ a difference between .1 and .5 km^2")
print(str(len(df[(df[0]<1) & (df[0]>=.5)]))+ " precincts w/ a difference between .5 and 1 km^2")
print(str(len(df[(df[0]<2) & (df[0]>=1)]))+ " precincts w/ a difference between 1 and 2 km^2")
print(str(len(df[(df[0]<5) & (df[0]>=2)]))+ " precincts w/ a difference between 2 and 5 km^2")
print(str(len(df[(df[0]>=5)]))+ " precincts w/ a difference greater than 5 km^2")

In [ ]:
source_geoms[source_geoms["unique_ID"]=="089-GLENNWOOD"].plot()
vest_geoms[vest_geoms["unique_ID"]=="089-GLENNWOOD"].plot()
print((source_geoms[source_geoms["unique_ID"]=="089-GLENNWOOD"].area-vest_geoms[vest_geoms["unique_ID"]=="089-GLENNWOOD"].area)/10e6)
